# 养老金税收拓展模型

## 基本内容

### 处理数值模拟结果

In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
from sbx import SAC
# from utils.sac import SAC
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
# import matplotlib
# print(matplotlib.matplotlib_fname())
# raise
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')
'''
pensionfund-v5-rural: 状态变量：财富，上期工资持续冲击，基本养老金个人账户余额, 城乡保缴费年限, 个人养老金账户余额，年龄，
'''
register(
id='pensionfund-v5-rural',                                # call it whatever you want
entry_point='v5_pensionfund_rural:PensionFundEnv', # module_name:class_name
)

''''
pensionfund-v5-nonrural: 状态变量：财富，上期工资持续冲击，个人养老金账户余额，年龄
'''
register(
    id='pensionfund-v5-nonrural',                                # call it whatever you want
    entry_point='v5_pensionfund_nonrural:PensionFundEnv', # module_name:class_name
)

distf = 0.95
# 将变量处理为终身路径
def var_path(vars, eval_episodes, env):
    max_length = max(len(path) for path in vars)
    var_path_array = np.full((eval_episodes, max_length), np.nan)
    for i, path in enumerate(vars):
        var_path_array[i, :len(path)] = path
    age_array = np.arange(env.get_attr('tb')[0], env.get_attr('tb')[0] + max_length)
    full_var_path = np.column_stack((age_array, var_path_array.T))
    var_path_df = pd.DataFrame(full_var_path,
                                    columns=['年龄'] + [f'个体{i+1}' for i in range(eval_episodes)])
    return var_path_df

def evaluate_model_rl(model_path,env_name,eval_episodes=1000, eval_seed=3687851522,params=None):


    # try:
    #     with open(os.path.join(current_path, log_dir, model_name.split('//')[0]) +"//params.json", 'r') as file:
    #         params = json.load(file)   
    # except:
    # if 'cocco' in idx:
    #     env = make_vec_env("cocco-rl-norm",seed=1, n_envs=1, env_kwargs={'params': params})
    # else:
    # env = make_vec_env("pensionfund-v3",seed=1, n_envs=1, env_kwargs={'params': params})````````
    env = make_vec_env(env_name,seed=1, n_envs=1, env_kwargs={'params':params},\
                            monitor_kwargs={'discount_factor':distf})
    

    env = VecNormalize.load(os.path.join(model_path,"best_model_vec_env.pkl"), env)
    # env.obs_rms.mean
    # env.obs_rms.var
    
    model = SAC.load(os.path.join(model_path, "best_model"), env=env)

    

    # env = make_vec_env("pensionfund-v1",seed=eval_seed, n_envs=1, env_kwargs={'params': None})
    
    # env = VecNormalize.load(os.path.join(current_path, log_dir, idx) + "//eval_env.pkl",env)

    env = make_vec_env(env_name,seed=eval_seed, n_envs=1, env_kwargs={'params':params},\
                            monitor_kwargs={'discount_factor': distf})         
    env = VecNormalize(env, norm_obs=False, norm_reward=False) # 评估不需要norm_obs


    idx = model_path.split('//')[0]
    # model = DDPG.load(os.path.join(log_dir, idx) + "//latest_model", env)



    # from tqdm import tqdm
    pbar = tqdm(total=eval_episodes, desc=f'模拟进度 {idx}')
    all_trajectories = []
    for episode in range(eval_episodes):       
        pbar.update(1)
        env.seed(seed=eval_seed+episode)
        obs = env.reset()    
        info = env.envs[0].env.env.env.info
        done = False
        agent_trajectory = {
            'raw_income':[],
            'states': [obs],
            'actions': [],
            'rewards': [],
            'infos': [info],
            'consumption': [],
            'norm_age':[obs[-1]],
            'tda_pen_amount':[],
            'basic_pen_amount':[],
            'tax':[],
            'tda_tax_savings':[],
            'basic_tax_savings':[],
            'tda_tax':[],
            'age':[info['age']],
            'cash':[obs[0][0]],
            'tda_pen_balance':[info['个人养老金实际余额'][0]],
            'basic_pen_balance':[info['基本养老金个人账户余额'][0] if '基本养老金个人账户余额' in info else np.nan]
        }
        while not done:
            action, _ = model.predict(model.env.normalize_obs(obs), deterministic=True) 
            # action[0,2] = 0

            obs, reward, done, info = env.step(action)               
            # 记录轨迹
            # if info[0]['status'] == 'working':
            agent_trajectory['raw_income'].append(info[0]['basic_income'])
            agent_trajectory['age'].append(info[0]['age'][0])

            agent_trajectory['states'].append(obs)
            agent_trajectory['actions'].append(info[0]['real_actions'])
            agent_trajectory['rewards'].append(reward)
            agent_trajectory['consumption'].append(info[0]['消费'])
            agent_trajectory['infos'].append(info[0])
            agent_trajectory['norm_age'].append(obs[0][-1])
            agent_trajectory['cash'].append(obs[0][0])
            agent_trajectory['tda_pen_amount'].append(info[0]['每年个人养老金购买额(实际)'][0] if '每年个人养老金购买额(实际)' in info[0] else np.nan)
            agent_trajectory['basic_pen_amount'].append(info[0]['每年基本养老金购买额(实际)'][0] if '每年基本养老金购买额(实际)' in info[0] else np.nan)
            agent_trajectory['tda_pen_balance'].append(info[0]['个人养老金实际余额'][0])
            agent_trajectory['basic_pen_balance'].append(info[0]['基本养老金个人账户余额'][0] if '基本养老金个人账户余额' in info[0] else np.nan)
            agent_trajectory['tax'].append(info[0]['每年收入应缴税额(万元)'][0] if '每年收入应缴税额(万元)' in info[0] else np.nan)
            agent_trajectory['tda_tax_savings'].append(info[0]['个人养老金退税额'][0] if '个人养老金退税额' in info[0] else np.nan)
            agent_trajectory['basic_tax_savings'].append(info[0]['基本养老金退税额'][0] if '基本养老金退税额' in info[0] else np.nan)
            agent_trajectory['tda_tax'].append(info[0]['当期个人养老金缴税额'][0] if '当期个人养老金缴税额' in info[0] else np.nan)

        all_trajectories.append(agent_trajectory)
    pbar.close()
        
    # 计算所有agent的reward统计信息
    all_age = np.concatenate([traj['age'] for traj in all_trajectories])
    # import pandas as pd
    # age_income = pd.DataFrame({'age':all_age,'income':all_income})
    # mean_income_by_age = age_income.groupby('age')['income'].mean()
    action_set = [traj['actions'] for traj in all_trajectories]
    max_length = max(len(path) for path in action_set)
    unique_age = np.arange(env.get_attr('tb')[0], env.get_attr('td')[0] + 1)[:max_length]
    num_action = np.shape(action_set[0])[1]
    aligned_actions = np.full((max_length, num_action, eval_episodes), np.nan)
    for i, actions in enumerate(action_set):
        aligned_actions[:len(actions), :, i] = actions


    data = {'income_history_df':var_path(list([traj['raw_income'] for traj in all_trajectories]), eval_episodes, env),
            'wealth_history_df':var_path(list([traj['cash'] for traj in all_trajectories]), eval_episodes, env),
            'tda_pen_balance_history_df':var_path(list([traj['tda_pen_balance'] for traj in all_trajectories]), eval_episodes, env),
            'basic_pen_balance_history_df':var_path(list([traj['basic_pen_balance'] for traj in all_trajectories]), eval_episodes, env),
          'aligned_actions':aligned_actions,
          'consum_history_df':var_path(list([traj['consumption'] for traj in all_trajectories]), eval_episodes, env),
          'tda_pen_amount_history_df':var_path(list([traj['tda_pen_amount'] for traj in all_trajectories]), eval_episodes, env),
          'basic_pen_amount_history_df':var_path(list([traj['basic_pen_amount'] for traj in all_trajectories]), eval_episodes, env),
          'tax_history_df':var_path(list([traj['tax'] for traj in all_trajectories]), eval_episodes, env),
          'tda_tax_savings_history_df':var_path(list([traj['tda_tax_savings'] for traj in all_trajectories]), eval_episodes, env),
          'basic_tax_savings_history_df':var_path(list([traj['basic_tax_savings'] for traj in all_trajectories]), eval_episodes, env),
          'tda_tax_history_df':var_path(list([traj['tda_tax'] for traj in all_trajectories]), eval_episodes, env),
          'unique_age':unique_age}
    # 保存pension_balance_history_df
    # model_identifier = 'rl_' + idx.split('-', 1)[1] 
    # os.makedirs(eval_path + model_identifier + '//', exist_ok=True)
    
    # pension_balance_history_df.to_excel(eval_path + model_identifier + '//' + model_identifier + "_pension_balance_history.xlsx",index=False)
    
    return data








if __name__ == '__main__':

    env_model_path_set = {'pensionfund-v5-rural':'models//rural//pensionfund-v5-rural_run13//',
                        'pensionfund-v5-nonrural':'models//nonrural//pensionfund-v5-nonrural_run83//'}

    # 创建进程池
    # pool = mp.Pool()
    
    # # 循环执行evaluate_model
    eval_episodes = 10000
    eval_seed =   3687851522


    rural_data = evaluate_model_rl(env_model_path_set['pensionfund-v5-rural'],'pensionfund-v5-rural',eval_episodes=eval_episodes, eval_seed=eval_seed)
    nonrural_data = evaluate_model_rl(env_model_path_set['pensionfund-v5-nonrural'],'pensionfund-v5-nonrural',eval_episodes=eval_episodes, eval_seed=eval_seed)

    
    save_data = {'rural_data':rural_data,
                 'nonrural_data':nonrural_data}
    with open('simu_data/1w_data.pkl', 'wb') as f:
        pickle.dump(save_data, f)

### 图p1：财富和养老金账户余额随年龄分布情况

In [ ]:

import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
os.environ['KMP_DUPLICATE_LIB_OK']='True'


if __name__ == '__main__':


    # # 循环执行evaluate_model
    # 读取数据
    with open('simu_data/1w_data.pkl', 'rb') as f:
        save_data = pickle.load(f)
    rural_data = save_data['rural_data']
    nonrural_data = save_data['nonrural_data']

    # 设置字体路径
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用简体中文字体
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题


    fig, axs =plt.subplots(1, 3, figsize=(9, 3))  # （宽度，高度）# （宽度，高度）
    '''
    第一张子图为生涯平均wealth路径比较
    '''
    nonrural_median = nonrural_data['wealth_history_df'].iloc[:, 1:].median(axis=1)
    nonrural_q10 = nonrural_data['wealth_history_df'].iloc[:, 1:].quantile(0.1, axis=1)
    nonrural_q90 = nonrural_data['wealth_history_df'].iloc[:, 1:].quantile(0.9, axis=1)
    rural_mean = rural_data['wealth_history_df'].iloc[:, 1:].mean(axis=1)
    rural_q10 = rural_data['wealth_history_df'].iloc[:, 1:].quantile(0.1, axis=1)
    rural_q90 = rural_data['wealth_history_df'].iloc[:, 1:].quantile(0.9, axis=1)


    axs[0].plot(nonrural_data['wealth_history_df']['年龄'], nonrural_median, color='blue', label='城镇')
    axs[0].plot(rural_data['wealth_history_df']['年龄'], rural_mean, color='orange', label='城乡')

    axs[0].fill_between(nonrural_data['wealth_history_df']['年龄'], nonrural_q10, nonrural_q90, color='blue', alpha=0.2,label='Q10-Q90')
    axs[0].fill_between(rural_data['wealth_history_df']['年龄'], rural_q10, rural_q90, color='orange', alpha=0.2)
    axs[0].legend()
    axs[0].set_title('(a) 财富')
    axs[0].set_xlabel('年龄')
    axs[0].set_ylabel('万元')
    axs[0].grid(True, which='both', linestyle='--', linewidth=0.5)  # 添加次要网格

    '''
    第二张子图为生涯个人养老金pension balance路径
    '''
    nonrural_pension_median = nonrural_data['tda_pen_balance_history_df'].iloc[:, 1:].median(axis=1)
    nonrural_pension_q10 = nonrural_data['tda_pen_balance_history_df'].iloc[:, 1:].quantile(0.1, axis=1)
    nonrural_pension_q90 = nonrural_data['tda_pen_balance_history_df'].iloc[:, 1:].quantile(0.9, axis=1)


    rural_pension_median = rural_data['tda_pen_balance_history_df'].iloc[:, 1:].median(axis=1)
    rural_pension_q10 = rural_data['tda_pen_balance_history_df'].iloc[:, 1:].quantile(0.1, axis=1)
    rural_pension_q90 = rural_data['tda_pen_balance_history_df'].iloc[:, 1:].quantile(0.9, axis=1)

    axs[1].plot(nonrural_data['tda_pen_balance_history_df']['年龄'], nonrural_pension_median, color='blue',label='城镇')
    axs[1].plot(rural_data['tda_pen_balance_history_df']['年龄'], rural_pension_median, color='orange',label='城乡')

    axs[1].fill_between(nonrural_data['tda_pen_balance_history_df']['年龄'], nonrural_pension_q10, nonrural_pension_q90, color='blue', alpha=0.2)
    axs[1].fill_between(rural_data['tda_pen_balance_history_df']['年龄'], rural_pension_q10, rural_pension_q90, color='orange', alpha=0.2)

    axs[1].set_title('(b) 个人养老金账户余额')
    
    axs[1].set_xlabel('年龄')
    axs[1].grid(True, which='both', linestyle='--', linewidth=0.5)  # 添加次要网格

    '''
    第3张子图为rural的基本养老金个人账户余额路径
    '''  
    rural_basic_pen_median = rural_data['basic_pen_balance_history_df'].iloc[:, 1:].median(axis=1)
    rural_basic_pen_q10 = rural_data['basic_pen_balance_history_df'].iloc[:, 1:].quantile(0.1, axis=1)
    rural_basic_pen_q90 = rural_data['basic_pen_balance_history_df'].iloc[:, 1:].quantile(0.9, axis=1)

    axs[2].plot(rural_data['basic_pen_balance_history_df']['年龄'], rural_basic_pen_median, color='orange',label='城乡')

    axs[2].fill_between(rural_data['basic_pen_balance_history_df']['年龄'], rural_basic_pen_q10, rural_basic_pen_q90, color='orange', alpha=0.2)

    axs[2].set_title('(c) 城乡保个人账户余额')
    axs[2].set_xlabel('年龄')
    axs[2].grid(True, which='both', linestyle='--', linewidth=0.5)  # 添加次要网格

    plt.tight_layout()
    plt.savefig('./figs/p1-wealth_pension_path.png',dpi=500)





### 图p2：不同收入阶层和不同年龄段下个人养老金投资比例和金额

In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import seaborn as sns
import matplotlib.pyplot as plt

if __name__ == '__main__':
    
    # # 读取数据

    with open('simu_data/1w_data.pkl', 'rb') as f:
        save_data = pickle.load(f)
    rural_data = save_data['rural_data']
    nonrural_data = save_data['nonrural_data']

    unique_age = rural_data['unique_age']


    ''' 
    （1）城乡居民个人养老金购买比例
    '''
    # 定义收入分位数和年龄段的分组
    income_bins = np.array([
        [0, 2],
        [2, 3],    
        [3, 4],
        [4, 5],
        [5, 10],
    ])
    age_bins = np.array([
        [18, 25],
        [26, 30],
        [31, 35],
        [36, 40],
        [41, 45],
        [46, 50],
        [51, 55],
        [56, 60],
    ])
    # 计算每个收入分组和年龄段的平均行动
    tr = 61
    agent_mean_income_before_tr = rural_data['income_history_df'].iloc[:tr, 1:].values.mean(axis=0) # 退休前平均年收
    mean_actions_by_income_and_age = []
    pension_action = rural_data['aligned_actions'][:, 3, :]
    tda_pension_amount = rural_data['tda_pen_amount_history_df'].iloc[:, 1:].values
    mean_pension_by_income_age_rural = np.full((len(income_bins),len(age_bins)), np.nan) 
    mean_pension_amount_by_income_age_rural = np.full((len(income_bins),len(age_bins)), np.nan) 

    idx_income = 0
    for start_income, end_income in income_bins: 
        agent_mask = (start_income <= agent_mean_income_before_tr) & (agent_mean_income_before_tr < end_income)
        # print(agent_mask.sum()/len(agent_mask))
        idx_age = 0
        for start_age, end_age in age_bins:
            age_mask = (unique_age >= start_age) & (unique_age <= end_age)
            mean_pension_by_income_age_rural[idx_income, idx_age] = np.nanmean(pension_action[age_mask][:, agent_mask])
            mean_pension_amount_by_income_age_rural[idx_income, idx_age] = np.nanmean(tda_pension_amount[age_mask][:, agent_mask])
            idx_age+=1
        idx_income+=1

    # 将mean_actions_by_age转换为DataFrame
    mean_pension_by_income_age_rural = pd.DataFrame(mean_pension_by_income_age_rural*100, columns=[f'{int(start)}-{int(end)}' for start, end in age_bins], index=[f'{int(start)}-{int(end)}' for start, end in income_bins])
    mean_pension_by_income_age_rural.index.name = '收入区间(万元)'
    mean_pension_amount_by_income_age_rural = pd.DataFrame(mean_pension_amount_by_income_age_rural*100, columns=[f'{int(start)}-{int(end)}' for start, end in age_bins], index=[f'{int(start)}-{int(end)}' for start, end in income_bins])
    mean_pension_amount_by_income_age_rural.index.name = '收入区间(万元)'

    ''' 
    （2）城镇居民个人养老金购买比例
    '''
    agent_mean_income_before_tr = nonrural_data['income_history_df'].iloc[:tr, 1:].values.mean(axis=0) # 退休前平均年收
    pension_action = nonrural_data['aligned_actions'][:, 2, :]
    tda_pension_amount = nonrural_data['tda_pen_amount_history_df'].iloc[:, 1:].values
    mean_pension_by_income_age_nonrural = np.full((len(income_bins),len(age_bins)), np.nan) 
    mean_pension_amount_by_income_age_nonrural = np.full((len(income_bins),len(age_bins)), np.nan) 

    idx_income = 0
    for start_income, end_income in income_bins: 
        agent_mask = (start_income <= agent_mean_income_before_tr) & (agent_mean_income_before_tr < end_income)
        idx_age = 0
        for start_age, end_age in age_bins:
            age_mask = (unique_age >= start_age) & (unique_age <= end_age)
            mean_pension_by_income_age_nonrural[idx_income, idx_age] = np.nanmean(pension_action[age_mask][:, agent_mask])
            mean_pension_amount_by_income_age_nonrural[idx_income, idx_age] = np.nanmean(tda_pension_amount[age_mask][:, agent_mask])
            idx_age+=1
        idx_income+=1

    mean_pension_by_income_age_nonrural = pd.DataFrame(mean_pension_by_income_age_nonrural*100, columns=[f'{int(start)}-{int(end)}' for start, end in age_bins], index=[f'{int(start)}-{int(end)}' for start, end in income_bins])
    mean_pension_by_income_age_nonrural.index.name = '收入区间(万元)'
    mean_pension_amount_by_income_age_nonrural = pd.DataFrame(mean_pension_amount_by_income_age_nonrural, columns=[f'{int(start)}-{int(end)}' for start, end in age_bins], index=[f'{int(start)}-{int(end)}' for start, end in income_bins])
    mean_pension_amount_by_income_age_nonrural.index.name = '收入区间(万元)'
    '''
    将表格以图的形式展现，并加上根据值的热力图
    '''


    plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用简体中文字体
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题


    # 创建热力图，第一个子图为城乡购买比例，第二个子图为城镇购买比例, 第三个子图为城乡购买金额，第四个子图为城镇购买金额
    fig, axes = plt.subplots(2, 2, figsize=(8, 6))
    sns.heatmap(mean_pension_by_income_age_rural, annot=True, cmap='Greys', fmt='.2f', cbar=False, ax=axes[0, 0])
    sns.heatmap(mean_pension_by_income_age_nonrural, annot=True, cmap='Greys', fmt='.2f', cbar=False, ax=axes[0, 1])
    sns.heatmap(mean_pension_amount_by_income_age_rural, annot=True, cmap='Greys', fmt='.2f', cbar=False, ax=axes[1, 0])
    sns.heatmap(mean_pension_amount_by_income_age_nonrural, annot=True, cmap='Greys', fmt='.2f', cbar=False, ax=axes[1, 1])
    # 行为年龄段为横坐标，收入分位数为纵坐标
    axes[0, 0].set_xlabel('')
    axes[0, 0].set_xticks([])
    axes[0, 0].set_ylabel('收入区间(万元)')
    axes[0, 0].set_title(r'(a) 城乡居民(%)')

    axes[0, 1].set_xlabel('')
    axes[0, 1].set_xticks([])
    axes[0, 1].set_ylabel('')
    axes[0, 1].set_yticks([])
    axes[0, 1].set_title(r'(b) 城镇居民(%)')

    axes[1, 0].set_xlabel('年龄段')
    axes[1, 0].set_ylabel('收入区间(万元)')
    axes[1, 0].set_title(r'(c) 城乡居民(百元)')

    axes[1, 1].set_xlabel('年龄段')
    axes[1, 1].set_ylabel('')
    axes[1, 1].set_yticks([])
    axes[1, 1].set_title(r'(d) 城镇居民(万元)')


    # 将两个子图的水平距离拉近一些
    plt.subplots_adjust(wspace=0.01)
    plt.tight_layout()
    plt.savefig('figs/p2-mean_pension_by_income_age.png', dpi=300)




### 图p7：不同收入和年龄段下城乡居民的城乡保配置比例和金额

In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import seaborn as sns
import matplotlib.pyplot as plt

if __name__ == '__main__':
    
    # # 读取数据

    with open('simu_data/1w_data.pkl', 'rb') as f:
        save_data = pickle.load(f)
    rural_data = save_data['rural_data']

    unique_age = rural_data['unique_age']


    ''' 
    （1）城乡居民个人养老金购买比例
    '''
    # 定义收入分位数和年龄段的分组
    income_bins = np.array([
        [1, 2],    
        [2, 3],
        [3, 4],
        [5, 6],
        [6, 10],
        [10, 30],
    ])
    age_bins = np.array([
        [18, 25],
        [26, 30],
        [31, 35],
        [36, 40],
        [41, 45],
        [46, 50],
        [51, 55],
        [56, 60],
    ])
    # 计算每个收入分组和年龄段的平均行动
    tr = 61
    agent_mean_income_before_tr = rural_data['income_history_df'].iloc[:tr, 1:].values.mean(axis=0) # 退休前平均年收
    mean_actions_by_income_and_age = []
    basic_pension_action = rural_data['aligned_actions'][:, 2, :]
    basic_pension_amount = rural_data['basic_pen_amount_history_df'].iloc[:, 1:].values
    mean_pension_by_income_age_rural = np.full((len(income_bins),len(age_bins)), np.nan) 
    mean_pension_amount_by_income_age_rural = np.full((len(income_bins),len(age_bins)), np.nan) 

    idx_income = 0
    for start_income, end_income in income_bins: 
        agent_mask = (start_income <= agent_mean_income_before_tr) & (agent_mean_income_before_tr < end_income)
        # print(agent_mask.sum()/len(agent_mask))
        idx_age = 0
        for start_age, end_age in age_bins:
            age_mask = (unique_age >= start_age) & (unique_age <= end_age)
            mean_pension_by_income_age_rural[idx_income, idx_age] = np.nanmean(basic_pension_action[age_mask][:, agent_mask])
            mean_pension_amount_by_income_age_rural[idx_income, idx_age] = np.nanmean(basic_pension_amount[age_mask][:, agent_mask])
            idx_age+=1
        idx_income+=1

    # 将mean_actions_by_age转换为DataFrame
    mean_pension_by_income_age_rural = pd.DataFrame(mean_pension_by_income_age_rural*100, columns=[f'{int(start)}-{int(end)}' for start, end in age_bins], index=[f'{int(start)}-{int(end)}' for start, end in income_bins])
    mean_pension_by_income_age_rural.index.name = '收入区间(万元)'
    mean_pension_amount_by_income_age_rural = pd.DataFrame(mean_pension_amount_by_income_age_rural, columns=[f'{int(start)}-{int(end)}' for start, end in age_bins], index=[f'{int(start)}-{int(end)}' for start, end in income_bins])
    mean_pension_amount_by_income_age_rural.index.name = '收入区间(万元)'


    '''
    将表格以图的形式展现，并加上根据值的热力图
    '''


    plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用简体中文字体
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题


    # 创建热力图，第一个子图为城乡购买比例，第二个子图为城镇购买比例, 第三个子图为城乡购买金额，第四个子图为城镇购买金额
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
    sns.heatmap(mean_pension_by_income_age_rural, annot=True, cmap='Greys', fmt='.2f', cbar=False, ax=axes[0])
    sns.heatmap(mean_pension_amount_by_income_age_rural, annot=True, cmap='Greys', fmt='.2f', cbar=False, ax=axes[1])
    # 行为年龄段为横坐标，收入分位数为纵坐标

    axes[0].set_xlabel('年龄段')
    axes[0].set_ylabel('收入区间(万元)')
    axes[0].set_title(r'(a) 城乡保比例(%)')

    axes[1].set_xlabel('年龄段')
    axes[1].set_ylabel('')
    axes[1].set_yticks([])
    axes[1].set_title(r'(b) 城乡保金额(万元)')


    # 将两个子图的水平距离拉近一些
    plt.subplots_adjust(wspace=0.01)
    plt.tight_layout()
    plt.savefig('figs/p7-mean_basic_pension_by_income_age.png', dpi=300)




### 图p6: 模拟收入和实际收入过程比较

In [ ]:
import csv
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
import gymnasium as gym
from statsmodels.tools.tools import add_constant
from linearmodels.panel.model import PanelOLS
from arch import arch_model
import numpy as np
import matplotlib.pyplot as plt
# Register this module as a gym environment. Once registered, the id is usable in gym.make().
from gymnasium.envs.registration import register
register(
    id='pensionfund-v5-rural',                                # call it whatever you want
    entry_point='v5_pensionfund_rural:PensionFundEnv', # module_name:class_name
)


'''
模拟数据
'''
def simulate_data(params,n_agents = 10000,age_range=(18,60)):
    # 测试环境
    # 定义参数字典


    env = gym.make('pensionfund-v5-rural', params=params)
    env = gym.wrappers.RecordEpisodeStatistics(env)

    # 模拟100个agent的路径
    

    # 存储所有agent的轨迹
    all_trajectories = []

    # 创建进度条
    from tqdm import tqdm
    pbar = tqdm(total=n_agents, desc='模拟进度')
    for agent in range(n_agents):
        # 重置环境获取初始状态
        obs, info = env.reset()
        done = False
        agent_trajectory = {
            'raw_income':[info['basic_income']],
            'states': [obs],
            'actions': [],
            'rewards': [],
            'infos': [info],
            'consumption': [],
            'norm_age':[obs[-1]],
            'age':[info['age']]
        }
        while not done:
            # 固定动作策略
            # 连续选择：消费比例，风险资产比例，购买城乡居民养老保险比例，购买个人养老金比例(若基本养老金个人账户余额不为0，则允许购买个人养老金)，个人养老金中购买风险资产比例
            action = np.clip(env.action_space.sample(),-1,2)
            action[0] = np.clip(action[0],1,1)
            action[2] = 0.1
            action[3] = 0.1
            
            # 执行动作
            obs, reward, done, _, info = env.step(action)
            
            # 记录轨迹
            # if info['status'] == 'working':
            agent_trajectory['raw_income'].append(info['basic_income'])
            agent_trajectory['age'].append(info['age'][0])

            agent_trajectory['states'].append(obs)
            agent_trajectory['actions'].append(action)
            agent_trajectory['rewards'].append(reward)
            agent_trajectory['consumption'].append(info['消费'])
            agent_trajectory['infos'].append(info)
            agent_trajectory['norm_age'].append(obs[-1])
            
        all_trajectories.append(agent_trajectory)
        pbar.update(1)
    pbar.close()
        
    # 计算所有agent的reward统计信息
    all_income = np.concatenate([traj['raw_income'] for traj in all_trajectories])
    all_age = np.concatenate([traj['age'] for traj in all_trajectories])
    import pandas as pd
    age_income = pd.DataFrame({'age':all_age,'income':all_income})
    mean_income_by_age_simu = age_income.groupby('age')['income'].mean()
    mean_income_by_age_simu.name = 'simu'
    mean_income_by_age_simu = mean_income_by_age_simu.loc[age_range[0]:age_range[1]]
    mean_income_by_age_f = pd.Series([np.exp(env.f(i)) for i in range(age_range[0],age_range[1]+1)], index=mean_income_by_age_simu.index)
    mean_income_by_age_f.name = 'f'
    return mean_income_by_age_simu,mean_income_by_age_f



'''
变量选择
pid: 个体唯一ID
age:年龄
qa301:户口类型
emp_income: 过去12个月所有工作（主要工作+一般工作）的税后工资性收入
retire: 是否退休 （“已退休”/“不适用”）
'''
n_sims = 200
var_list = ['pid','age','qa301','emp_income','retire']


# 读取数据
da_rural = pd.read_csv('./cfps_data/da_rural.csv', delimiter=",", encoding='gbk')
da_non_rural = pd.read_csv('./cfps_data/da_non_rural.csv', delimiter=",", encoding='gbk')

    # 设置字体路径
plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用简体中文字体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题
# 使用subplot，左图为城镇，右图为农村
fig, axs = plt.subplots(1, 2, figsize=(8, 3))
for da in [da_rural,da_non_rural]:
    '''
    真实数据
    '''
    data = da.set_index(['pid', 'year'])
    data = data.dropna() # 删除包含 NaN 的行
    # 删除'emp_income_before_tax'小于10的行
    data = data.loc[data['emp_income_before_tax'] >= 10,:]
    data['age'] = data['age'].apply(lambda x: int(x) if x.isdigit() else np.nan)
    '''
    城乡69岁仍未退休的样本有32个，70岁仍未退休的样本有17个
    因此城乡居民取到69岁,此后为拟合数据
    '''
    if da is da_rural:
        data = data.loc[(data['age']>=18) & (data['age']<=75),:] 
    else:
        data = data.loc[(data['age']>=18) & (data['age']<=60),:] 

    data = data.groupby('pid').filter(lambda x: x['age'].nunique() >= 2)

    mean_income_by_age_real = data.groupby('age')['emp_income_before_tax'].mean()/10000
    mean_income_by_age_real.name = 'real'
    '''
    模拟数据
    '''
    if da is da_non_rural:
        # 城镇参数
        params = {
            'aa': -1.32157880e+01,
            'b1': 1.34951371e+00,
            'b2': -4.33632834e-02,
            'b3': 5.85163503e-04,
            'b4':-2.81009579e-06,
            'ar': 0.917415,
            'smay': np.sqrt(0.167829), # 暂时性shock的标准差
            'smav': np.sqrt(0.084805), # 永久性shock的标准差
        }
    else:
        # 城乡参数
        params = {
            'aa': -1.09533849e+01,
            'b1': 1.19983877e+00,
            'b2': -4.18823630e-02,
            'b3': 6.23094506e-04,
            'b4':-3.40219380e-06,
            'ar': 0.838679,
            'smay': np.sqrt(0.269354), # 暂时性shock的标准差
            'smav': np.sqrt(0.155491), # 永久性shock的标准差
        }
    mean_income_by_age_simu,mean_income_by_age_f = simulate_data(params,n_agents = n_sims, age_range=(18,100) if da is da_rural else (18,60))

    merge_income = pd.merge(mean_income_by_age_real, mean_income_by_age_simu, left_index=True, right_index=True, how='outer')
    merge_income = pd.merge(merge_income, mean_income_by_age_f, left_index=True, right_index=True, how='outer')
    merge_income.columns = ['real','simu','f']
    print(merge_income)


    if da is da_non_rural:
        ax = axs[0]
        ax.set_title('城镇')
    else:
        ax = axs[1]
        ax.set_title('城乡')
    ax.plot(merge_income.index, merge_income['real'], label='CFPS', marker='o',markersize=3)
    ax.plot(merge_income.index, merge_income['simu'], label='模拟', marker='o',markersize=3)
    ax.plot(merge_income.index, merge_income['f'], label='$\hat f_t$', marker='o',markersize=3)
    if da is da_rural:
        ax.legend()
    ax.set_xlabel('年龄')
    if da is da_non_rural:
        ax.set_ylabel('收入(万元)')
    # 添加grid
    ax.grid(True, linestyle='--', alpha=1)
# 保存为png

plt.tight_layout()
plt.savefig('figs/p6-income_path_comparison.png',dpi=300)



### 表t1：不同年龄段下居民财富配置情况

In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
os.environ['KMP_DUPLICATE_LIB_OK']='True'

if __name__ == '__main__':

    #读取数据
    with open('simu_data/1w_data.pkl', 'rb') as f:
        data = pickle.load(f)
    rural_data = data['rural_data']
    nonrural_data = data['nonrural_data']
    
    unique_age = rural_data['unique_age']


    ''' 
    rural
    '''
    age_bins = np.array([
        [20, 29],
        [30, 39],
        [40, 49],
        [50, 59],
        [60, 65],
        [66, 74],
        [75, 84],
        [85, 94],
        [95, 100]
    ])
    mean_actions_by_age = []
    for start_age, end_age in age_bins:
        mask = (unique_age >= start_age) & (unique_age <= end_age)
        mean_actions = np.nanmean(rural_data['aligned_actions'][mask], axis=(0, 2)) # 转换为百分比
        mean_actions_by_age.append(mean_actions)

    mean_actions_by_age = np.array(mean_actions_by_age)

    # 将mean_actions_by_age转换为DataFrame
    mean_actions_by_age_rural = pd.DataFrame(mean_actions_by_age, columns=['消费', '风险资产', '城乡保','个人养老金', '个人养老金风险资产'])
    # 在第一列增加年龄段，格式为：20-29，最后的年龄段为90-99
    mean_actions_by_age_rural.insert(0, '年龄段', [f'{int(start)}-{int(end)}' for start, end in age_bins])
    # 保存为excel

    '''
    nonrual
    '''
    # 计算每10岁一次的actions平均值:20-29,30-39,40-49,50-59,60-69,70-79,80-89,90-99
    mean_actions_by_age = []
    for start_age, end_age in age_bins:
        mask = (unique_age >= start_age) & (unique_age <= end_age)
        mean_actions = np.nanmean(nonrural_data['aligned_actions'][mask], axis=(0, 2)) # 转换为百分比
        mean_actions_by_age.append(mean_actions)

    mean_actions_by_age = np.array(mean_actions_by_age)
    # 将mean_actions_by_age转换为DataFrame
    mean_actions_by_age_nonrural = pd.DataFrame(mean_actions_by_age, columns=['消费', '风险资产', '个人养老金', '个人养老金风险资产'])
    # 在第一列增加年龄段，格式为：20-29，最后的年龄段为90-99
    mean_actions_by_age_nonrural.insert(0, '年龄段', [f'{int(start)}-{int(end)}' for start, end in age_bins])
    # 保存为excel

    '''
    绘制表格
    '''
    # 创建一个包含所有数据的DataFrame：
    # 第一列为年龄段，第2列、第3列分别为naive和rl的消费，第4列、第5列分别为naive和rl的风险资产，第6列分为rl的个人养老金购买比例，第7列为rl的个人养老金风险资产比例

    '''
    城乡居民：cash->基本养老金比例->个人养老金比例->消费比例->风险资产比例
    城镇职工：cash->基本养老金比例(28\%)->个人养老金比例->消费比例->风险资产比例
    '''
    nonrural={}
    nonrural['消费'] = (1- 0.28)*(1-mean_actions_by_age_nonrural['个人养老金'])*mean_actions_by_age_nonrural['消费']
    nonrural['股票'] = (1- 0.28)*(1-mean_actions_by_age_nonrural['个人养老金'])*(1-mean_actions_by_age_nonrural['消费'])*mean_actions_by_age_nonrural['风险资产']
    nonrural['债券'] = (1- 0.28)*(1-mean_actions_by_age_nonrural['个人养老金'])*(1-mean_actions_by_age_nonrural['消费'])*(1-mean_actions_by_age_nonrural['风险资产'])
    nonrural['个人养老金'] = (1- 0.28)*mean_actions_by_age_nonrural['个人养老金']
    nonrural['个人养老金股票'] = (1- 0.28)*mean_actions_by_age_nonrural['个人养老金']*mean_actions_by_age_nonrural['个人养老金风险资产']
    nonrural['个人养老金债券'] = (1- 0.28)*mean_actions_by_age_nonrural['个人养老金']*(1-mean_actions_by_age_nonrural['个人养老金风险资产'])



    rural={}
    rural['消费'] = (1-mean_actions_by_age_rural['城乡保'])*mean_actions_by_age_rural['消费']
    rural['股票'] = (1-mean_actions_by_age_rural['城乡保'])*(1-mean_actions_by_age_rural['消费'])*mean_actions_by_age_rural['风险资产']
    rural['债券'] = (1-mean_actions_by_age_rural['城乡保'])*(1-mean_actions_by_age_rural['消费'])*(1-mean_actions_by_age_rural['风险资产'])
    rural['个人养老金'] = (1-mean_actions_by_age_rural['城乡保'])*mean_actions_by_age_rural['个人养老金']
    rural['个人养老金股票'] = (1-mean_actions_by_age_rural['城乡保'])*mean_actions_by_age_rural['个人养老金']*mean_actions_by_age_rural['个人养老金风险资产']
    rural['个人养老金债券'] = (1-mean_actions_by_age_rural['城乡保'])*mean_actions_by_age_rural['个人养老金']*(1-mean_actions_by_age_rural['个人养老金风险资产'])
    rural['城乡保'] = mean_actions_by_age_rural['城乡保']
    

    combined_data = pd.DataFrame({
        '年龄段': mean_actions_by_age_rural['年龄段'],
        '城乡_消费': rural['消费']* 100,
        '城镇_消费': nonrural['消费']* 100,
        '城乡_股票': rural['股票']* 100,
        '城镇_股票': nonrural['股票']* 100,
        '城乡_债券': rural['债券']* 100,
        '城镇_债券': nonrural['债券']* 100,
        '城乡_个人养老金': rural['个人养老金']* 100,
        '城乡_个人养老金股票': rural['个人养老金股票']* 100,
        '城乡_个人养老金债券': rural['个人养老金债券']* 100,
        '城镇_个人养老金': nonrural['个人养老金']* 100,
        '城镇_个人养老金股票': nonrural['个人养老金股票']* 100,
        '城镇_个人养老金债券': nonrural['个人养老金债券']* 100,
        '城乡_城乡保': rural['城乡保']* 100,
    })

    '''
    导出为latex表格
    '''
    # 保留小数点后3位
    combined_data = combined_data.round(3)

    # 两个消费列为multicolumn,两个风险资产列为multicolumn, 仅显示到小数点后3位
    table_tex = combined_data.to_latex(index=False, multicolumn=True, multicolumn_format='c', float_format='%.2f')

    table_tex = table_tex.split(r'\midrule', 1)[1]  # 去除\midrule以前的内容
    header = r'''
\begin{table}[htbp]
	\caption{不同年龄段下居民财富配置情况}
	\label{t1-action_by_age}
\centering
\makebox[\textwidth][c]{
\begin{threeparttable}
\begin{tabular}{C{1.5cm}*{12}{C{1cm}}C{1cm}}
	\toprule
	\multicolumn{1}{c}{\multirow{2}{*}{年龄段}} & \multicolumn{2}{c}{消费} & \multicolumn{2}{c}{股票} & \multicolumn{2}{c}{债券}&  \multicolumn{6}{c}{个人养老金} & \multicolumn{1}{c}{\multirow{2}{*}{城乡保}}\\ \cmidrule(r){2-3}\cmidrule(r){4-5}\cmidrule(r){6-7}\cmidrule(r){8-13}
	\multicolumn{1}{c}{} & 城乡 & 城镇 & 城乡 & 城镇 & 城乡 & 城镇   & 城乡 & 股票 & 债券 & 城镇 & 股票 & 债券 &  \multicolumn{1}{c}{}\\ \cmidrule(r){1-7}\cmidrule(r){8-10}\cmidrule(r){11-13}\cmidrule(r){14-14}
    '''
    ender = r'''\begin{footnotesize}
					注：表中数值均为比例(\%)； 对于城乡居民而言, 其消费+股票+债券+个人养老金+城乡保=100\%；对于城镇居民而言, 其消费+股票+债券+个人养老金+28\%的城镇保=100\%。
		\end{footnotesize}
        \end{threeparttable}}
        \end{table}
        '''
    table_tex = header + table_tex.replace('\n', ' ').replace('\\\\', '\\\\ \n') + '\n' + ender # 加入自定义表头(header和table_tex之间不要有空行)
    # 将table_tex导出为latex文件
    with open('table/t1-action_by_age.tex', 'w') as f:
        f.write(table_tex)
    print(table_tex)




### 表t6: 不同收入区间的纳税和节税结果

In [ ]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import os
import pickle
os.environ['KMP_DUPLICATE_LIB_OK']='True'

def nonrural_process(nonrural_data,tr,tb,income_bins,distf):
    '''
    城镇居民
    '''
    # 节税
    tda_tax_saving_nonrural_df = nonrural_data['tda_tax_savings_history_df'].iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    basic_tax_savings_nonrural_df = nonrural_data['basic_tax_savings_history_df'].iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    tda_tax_saving_distf = tda_tax_saving_nonrural_df.iloc[:tr-tb+1, :].values
    sum_tda_tax_saving_distf = np.nansum(tda_tax_saving_distf,axis=0)
    
    basic_tax_saving_distf = basic_tax_savings_nonrural_df.iloc[:tr-tb+1, :].values
    sum_basic_tax_saving_distf = np.nansum(basic_tax_saving_distf,axis=0)
    

    # 收入
    agent_mean_income_before_tr = np.nanmean(nonrural_data['income_history_df'].iloc[:tr-tb+1, 1:].values,axis=0) # 退休前平均年收
    # # # 保存到csv
    pd.DataFrame(agent_mean_income_before_tr).to_csv('nonrural_income_history_df.csv')
    temp_income_before_tr = nonrural_data['income_history_df'].iloc[:tr-tb+1, 1:]
    temp_dist_income_before_tr = temp_income_before_tr.apply(lambda x: x * distf ** np.arange(len(x))).values # 退休前折现收入
    agent_dist_sum_income_before_tr = np.nansum(temp_dist_income_before_tr,axis=0) # 退休前折现总收入

    # # 保存到csv
    # pd.DataFrame(agent_dist_sum_income_before_tr).to_csv('nonrural_income_history_df.csv')

    # 税收
    tax_nonrural_df = nonrural_data['tax_history_df'].iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    eff_tax = tax_nonrural_df.iloc[:tr-tb+1, :].values # 工作期税收
    eff_tax_dist = tax_nonrural_df.iloc[:tr-tb+1, :].apply(lambda x: x * distf ** np.arange(len(x))).values # 工作期折现税收
    agent_dist_sum_tax_before_tr = np.nansum(eff_tax_dist,axis=0) # 工作期折现总税收
    tda_tax_nonrural_df = nonrural_data['tda_tax_history_df'].iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    tda_tax = tda_tax_nonrural_df.iloc[tr-tb:, :].values
    sum_tda_tax = np.nansum(tda_tax,axis=0)
    

   # 养老金购买
    tda_pen_amount_nonrural_df = nonrural_data['tda_pen_amount_history_df'].iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    tda_pen_amount_distf = tda_pen_amount_nonrural_df.iloc[:tr-tb+1, :].values
    sum_tda_pen_amount_distf = np.nansum(tda_pen_amount_distf,axis=0)
    

    basic_pen_amount_nonrural_df = nonrural_data['basic_pen_amount_history_df'].iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    basic_pen_amount_distf = basic_pen_amount_nonrural_df.iloc[:tr-tb+1, :].values
    sum_basic_pen_amount_distf = np.nansum(basic_pen_amount_distf,axis=0)
    



    # 循环: 征税额、个人养老金购买额、基本养老金购买额、个人养老金节税额、基本养老金节税额
    tax_by_income = np.full((len(income_bins),3), np.nan) 
    tda_tax_by_income = np.full((len(income_bins),1), np.nan) 
    tda_pen_amount_by_income = np.full((len(income_bins),1), np.nan) 
    basic_pen_amount_by_income = np.full((len(income_bins),1), np.nan) 
    tda_tax_saving_by_income = np.full((len(income_bins),1), np.nan) 
    basic_tax_saving_by_income = np.full((len(income_bins),1), np.nan) 
    idx_income = 0
    for start_income, end_income in income_bins: 
        agent_mask = (np.nanpercentile(agent_mean_income_before_tr, start_income) <= agent_mean_income_before_tr) & (agent_mean_income_before_tr < np.nanpercentile(agent_mean_income_before_tr, end_income))

        print('收入区间：',start_income,end_income,'平均收入:',np.nanmean(agent_dist_sum_income_before_tr[agent_mask]))
        # 平均有效税率
        tax_by_income[idx_income,0] = np.nanmean(agent_dist_sum_tax_before_tr[agent_mask]/agent_dist_sum_income_before_tr[agent_mask])*100 
        # 平均缴税额现值
        tax_by_income[idx_income,1] = np.nanmean(agent_dist_sum_tax_before_tr[agent_mask]) 
        # 收入
        tax_by_income[idx_income,2] = np.nanmean(agent_dist_sum_income_before_tr[agent_mask]) 

        # 平均个人养老金购买额
        tda_pen_amount_by_income[idx_income,0] = np.nanmean(sum_tda_pen_amount_distf[agent_mask])
        # 平均基本养老金购买额
        basic_pen_amount_by_income[idx_income,0] = np.nanmean(sum_basic_pen_amount_distf[agent_mask])
        # 平均个人养老金节税额
        tda_tax_saving_by_income[idx_income,0] = np.nanmean(sum_tda_tax_saving_distf[agent_mask])
        # 平均基本养老金节税额
        basic_tax_saving_by_income[idx_income,0] = np.nanmean(sum_basic_tax_saving_distf[agent_mask])
        # 平均养老金缴税额
        tda_tax_by_income[idx_income,0] = np.nanmean(sum_tda_tax[agent_mask])
        
        idx_income+=1

    # 将mean_actions_by_age转换为DataFrame
    nonrural_data = pd.DataFrame(np.hstack((tax_by_income, tda_pen_amount_by_income, basic_pen_amount_by_income, tda_tax_saving_by_income, basic_tax_saving_by_income, tda_tax_by_income)), columns=['有效税率(\%)', '工作期总缴税额', '总收入', '个人养老金购买额', '基本养老金购买额', '个人养老金节税额', '基本养老金节税额', '养老金缴税额'], index=[f'{int(start)}-{int(end)}' for start, end in income_bins])
    nonrural_data.index.name = '收入区间'
    return nonrural_data

def rural_process(rural_data,tr,tb,income_bins,distf):
    '''
    城乡居民
    '''
    # 节税
    tda_tax_saving_rural_df = rural_data['tda_tax_savings_history_df'].iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    basic_tax_savings_rural_df = rural_data['basic_tax_savings_history_df'].iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    tda_tax_saving_distf = tda_tax_saving_rural_df.iloc[:tr-tb+1, :].values
    sum_tda_tax_saving_distf = np.nansum(tda_tax_saving_distf,axis=0)
    basic_tax_saving_distf = basic_tax_savings_rural_df.iloc[:tr-tb+1, :].values
    sum_basic_tax_saving_distf = np.nansum(basic_tax_saving_distf,axis=0)
    


    # 税收
    tax_rural_df = rural_data['tax_history_df'].iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    eff_tax = tax_rural_df.iloc[:tr-tb+1, :].values # 工作期税收
    eff_tax_dist = tax_rural_df.iloc[:tr-tb+1, :].apply(lambda x: x * distf ** np.arange(len(x))).values # 工作期折现税收
    agent_dist_sum_tax_before_tr = np.nansum(eff_tax_dist,axis=0) # 工作期折现总税收

    # 养老金购买
    tda_pen_amount_rural_df = rural_data['tda_pen_amount_history_df'].iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    tda_pen_amount_distf = tda_pen_amount_rural_df.iloc[:tr-tb+1, :].values
    sum_tda_pen_amount_distf = np.nansum(tda_pen_amount_distf,axis=0)
    
    basic_pen_amount_rural_df = rural_data['basic_pen_amount_history_df'].iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    basic_pen_amount_distf = basic_pen_amount_rural_df.iloc[:tr-tb+1, :].values
    sum_basic_pen_amount_distf = np.nansum(basic_pen_amount_distf,axis=0)

    # 养老金缴税
    tda_tax_rural_df = rural_data['tda_tax_history_df'].iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    tda_tax = tda_tax_rural_df.iloc[tr-tb:, :].values
    sum_tda_tax = np.nansum(tda_tax,axis=0)

    # 收入
    agent_mean_income_before_tr = np.nanmean(rural_data['income_history_df'].iloc[:tr-tb+1, 1:].values,axis=0) # 退休前平均年收
    # # 保存到csv
    pd.DataFrame(agent_mean_income_before_tr).to_csv('rural_income_history_df.csv')
    temp_income_before_tr = rural_data['income_history_df'].iloc[:tr-tb+1, 1:]
    temp_dist_income_before_tr = temp_income_before_tr.apply(lambda x: x * distf ** np.arange(len(x))).values # 退休前折现收入
    agent_dist_sum_income_before_tr = np.nansum(temp_dist_income_before_tr,axis=0) # 退休前折现总收入

    # # 保存到csv
    # pd.DataFrame(agent_dist_sum_income_before_tr).to_csv('rural_income_history_df.csv')

    # 循环: 征税额、个人养老金购买额、基本养老金购买额、个人养老金节税额、基本养老金节税额
    tax_by_income = np.full((len(income_bins),3), np.nan) 
    tda_tax_by_income = np.full((len(income_bins),1), np.nan) 
    tda_pen_amount_by_income = np.full((len(income_bins),1), np.nan) 
    basic_pen_amount_by_income = np.full((len(income_bins),1), np.nan) 
    tda_tax_saving_by_income = np.full((len(income_bins),1), np.nan) 
    basic_tax_saving_by_income = np.full((len(income_bins),1), np.nan) 
    idx_income = 0
    for start_income, end_income in income_bins: 
        agent_mask = (np.nanpercentile(agent_mean_income_before_tr, start_income) <= agent_mean_income_before_tr) & (agent_mean_income_before_tr < np.nanpercentile(agent_mean_income_before_tr, end_income))
        print('收入区间：',start_income,end_income,'平均收入:',np.nanmean(agent_dist_sum_income_before_tr[agent_mask]))
        # 平均有效税率
        tax_by_income[idx_income,0] = np.nanmean(agent_dist_sum_tax_before_tr[agent_mask]/agent_dist_sum_income_before_tr[agent_mask])*100 
        # 平均缴税额现值
        tax_by_income[idx_income,1] = np.nanmean(agent_dist_sum_tax_before_tr[agent_mask]) 
                # 收入
        tax_by_income[idx_income,2] = np.nanmean(agent_dist_sum_income_before_tr[agent_mask]) 
        # 平均个人养老金购买额
        tda_pen_amount_by_income[idx_income,0] = np.nanmean(sum_tda_pen_amount_distf[agent_mask])
        # 平均基本养老金购买额
        basic_pen_amount_by_income[idx_income,0] = np.nanmean(sum_basic_pen_amount_distf[agent_mask])
        # 平均个人养老金节税额
        tda_tax_saving_by_income[idx_income,0] = np.nanmean(sum_tda_tax_saving_distf[agent_mask])
        # 平均基本养老金节税额
        basic_tax_saving_by_income[idx_income,0] = np.nanmean(sum_basic_tax_saving_distf[agent_mask])
        # 平均养老金缴税额
        tda_tax_by_income[idx_income,0] = np.nanmean(sum_tda_tax[agent_mask])
        
        idx_income+=1

    # 将mean_actions_by_age转换为DataFrame
    rural_data = pd.DataFrame(np.hstack((tax_by_income, tda_pen_amount_by_income, basic_pen_amount_by_income, tda_tax_saving_by_income, basic_tax_saving_by_income, tda_tax_by_income)), columns=['有效税率(\%)', '工作期总缴税额', '总收入', '个人养老金购买额', '基本养老金购买额', '个人养老金节税额', '基本养老金节税额', '养老金缴税额'], index=[f'{int(start)}-{int(end)}' for start, end in income_bins])
    rural_data.index.name = '收入区间'
    return rural_data

if __name__ == '__main__':

    #读取数据
    with open('simu_data/1w_data.pkl', 'rb') as f:
        data = pickle.load(f)
    rural_data = data['rural_data']
    nonrural_data = data['nonrural_data']
    
    unique_age = rural_data['unique_age']
    tr = 61
    tb=18
    # 定义收入分位数分组
    income_bins = np.array([
        [0, 15],    
        [15, 30],
        [30, 45],
        [45, 60],
        [60, 75],
        [75, 90],
        [90, 100],
    ])



    # 将数值转变为折现数值
    distf = 0.95
 

    nonrural_data = nonrural_process(nonrural_data,tr,tb,income_bins,distf)
    rural_data = rural_process(rural_data,tr,tb,income_bins,distf)
    
  
    
    '''
    导出为latex表格
    '''
    # 保留小数点后3位
    nonrural_data.reset_index(drop=False, inplace=True)
    # 仅显示到小数点后3位
    nonrural_table_tex = nonrural_data.to_latex(index=False, multicolumn=True,float_format='%.3f')
    nonrural_table_tex = nonrural_table_tex.split(r'\midrule', 1)[1]  # 去除\midrule以前的内容
    # 去除\bottomrule以后的内容
    nonrural_table_tex = nonrural_table_tex.split(r'\bottomrule', 1)[0]
    # 将表格中的NaN替换为'--'
    nonrural_table_tex = nonrural_table_tex.replace('NaN', '--')



    rural_data.reset_index(drop=False, inplace=True)
    rural_table_tex = rural_data.to_latex(index=False, multicolumn=True,float_format='%.3f')
    rural_table_tex = rural_table_tex.split(r'\midrule', 1)[1]  # 去除\midrule以前的内容
    # 将表格中的NaN替换为'--'
    rural_table_tex = rural_table_tex.replace('NaN', '--')



    header = r'''
    \begin{table}[htbp]
        \caption{不同收入区间的纳税和购买个人养老金节税结果}
        \label{t7-tax_saving_result}
    \centering
    \begin{threeparttable}
    \begin{tabular}{>{\centering\arraybackslash}p{1.5cm} >{\centering\arraybackslash}p{1.5cm}  >{\centering\arraybackslash}p{1.5cm}>{\centering\arraybackslash}p{1.5cm} >{\centering\arraybackslash}p{1.5cm} >{\centering\arraybackslash}p{1.5cm} >{\centering\arraybackslash}p{1.5cm} >{\centering\arraybackslash}p{1.5cm} >{\centering\arraybackslash}p{1.5cm}} \toprule 收入区间 & 有效税率(\%) & 总缴税额  & 总收入 & 个人养老金 & 基本养老金 & 个人养老金节税额 & 基本养老金节税额 & 养老金缴税额 \\ 
    \midrule
    \multicolumn{9}{c}{\underline{城镇居民}} \\
    '''
    ender = r'''\begin{footnotesize}
        注：除有效税率外，表中所有数额均为折现至第一期的生涯总数额；有效税率为总缴税额除以总收入；城镇居民的基本养老金为固定缴纳的每年收入的28\%。
		\end{footnotesize}
        \end{threeparttable}
        \end{table}
        '''
    
    table_tex = header + nonrural_table_tex.replace('\n', ' ').replace('\\\\', '\\\\ \n') + r'\multicolumn{9}{c}{\underline{城乡居民}} \\' + rural_table_tex.replace('\n', ' ').replace('\\\\', '\\\\ \n') + '\n' +  ender # 加入自定义表头和表尾(header和table_tex之间不要有空行)
    # 将table_tex导出为latex文件
    with open('table/t6-tax_saving_result.tex', 'w') as f:
        f.write(table_tex)
    print(table_tex)
    








### 表t7: 设置个人养老金缴费上限的效用损失

In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
from sbx import SAC
# from utils.sac import SAC
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
# import matplotlib
# print(matplotlib.matplotlib_fname())
# raise
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))

# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')
'''
pensionfund-v5-rural: 状态变量：财富，上期工资持续冲击，基本养老金个人账户余额, 城乡保缴费年限, 个人养老金账户余额，年龄，
'''
register(
id='pensionfund-v5-rural',                                # call it whatever you want
entry_point='v5_pensionfund_rural:PensionFundEnv', # module_name:class_name
)

''''
pensionfund-v5-nonrural: 状态变量：财富，上期工资持续冲击，个人养老金账户余额，年龄
'''
register(
    id='pensionfund-v5-nonrural',                                # call it whatever you want
    entry_point='v5_pensionfund_nonrural:PensionFundEnv', # module_name:class_name
)

distf = 0.95
# 将变量处理为终身路径
def var_path(vars, eval_episodes, env):
    max_length = max(len(path) for path in vars)
    var_path_array = np.full((eval_episodes, max_length), np.nan)
    for i, path in enumerate(vars):
        var_path_array[i, :len(path)] = path
    age_array = np.arange(env.get_attr('tb')[0], env.get_attr('tb')[0] + max_length)
    full_var_path = np.column_stack((age_array, var_path_array.T))
    var_path_df = pd.DataFrame(full_var_path,
                                    columns=['年龄'] + [f'个体{i+1}' for i in range(eval_episodes)])
    return var_path_df

def evaluate_model_rl(model_path,env_name,eval_episodes=1000, eval_seed=3687851522,params=None):


    # try:
    #     with open(os.path.join(current_path, log_dir, model_name.split('//')[0]) +"//params.json", 'r') as file:
    #         params = json.load(file)   
    # except:
    # if 'cocco' in idx:
    #     env = make_vec_env("cocco-rl-norm",seed=1, n_envs=1, env_kwargs={'params': params})
    # else:
    # env = make_vec_env("pensionfund-v3",seed=1, n_envs=1, env_kwargs={'params': params})````````
    env = make_vec_env(env_name,seed=1, n_envs=1, env_kwargs={'params':params},\
                            monitor_kwargs={'discount_factor':distf})
    

    env = VecNormalize.load(os.path.join(model_path,"best_model_vec_env.pkl"), env)
    # env.obs_rms.mean
    # env.obs_rms.var
    
    model = SAC.load(os.path.join(model_path, "best_model"), env=env)

    

    # env = make_vec_env("pensionfund-v1",seed=eval_seed, n_envs=1, env_kwargs={'params': None})
    
    # env = VecNormalize.load(os.path.join(current_path, log_dir, idx) + "//eval_env.pkl",env)

    env = make_vec_env(env_name,seed=eval_seed, n_envs=1, env_kwargs={'params':params},\
                            monitor_kwargs={'discount_factor': distf})         
    env = VecNormalize(env, norm_obs=False, norm_reward=False) # 评估不需要norm_obs


    idx = model_path.split('//')[0]
    # model = DDPG.load(os.path.join(log_dir, idx) + "//latest_model", env)



    # from tqdm import tqdm
    pbar = tqdm(total=eval_episodes, desc=f'模拟进度 {idx}')
    all_trajectories = []
    for episode in range(eval_episodes):       
        pbar.update(1)
        env.seed(seed=eval_seed+episode)
        obs = env.reset()    
        info = env.envs[0].env.env.env.info
        done = False
        agent_trajectory = {
            'raw_income':[],
            'states': [obs],
            'actions': [],
            'rewards': [],
            'infos': [info],
            'consumption': [],
            'norm_age':[obs[-1]],
            'tda_pen_amount':[],
            'basic_pen_amount':[],
            'tax':[],
            'tda_tax_savings':[],
            'basic_tax_savings':[],
            'tda_tax':[],
            'age':[info['age']],
            'cash':[obs[0][0]],
            'tda_pen_balance':[info['个人养老金实际余额'][0]],
            'basic_pen_balance':[info['基本养老金个人账户余额'][0] if '基本养老金个人账户余额' in info else np.nan]
        }
        while not done:
            action, _ = model.predict(model.env.normalize_obs(obs), deterministic=True) 
            # action[0,2] = 0

            obs, reward, done, info = env.step(action)               
            # 记录轨迹
            # if info[0]['status'] == 'working':
            agent_trajectory['raw_income'].append(info[0]['basic_income'])
            agent_trajectory['age'].append(info[0]['age'][0])

            agent_trajectory['states'].append(obs)
            agent_trajectory['actions'].append(info[0]['real_actions'])
            agent_trajectory['rewards'].append(reward[0])
            agent_trajectory['consumption'].append(info[0]['消费'])
            agent_trajectory['infos'].append(info[0])
            agent_trajectory['norm_age'].append(obs[0][-1])
            agent_trajectory['cash'].append(obs[0][0])
            agent_trajectory['tda_pen_amount'].append(info[0]['每年个人养老金购买额(实际)'][0] if '每年个人养老金购买额(实际)' in info[0] else np.nan)
            agent_trajectory['basic_pen_amount'].append(info[0]['每年基本养老金购买额(实际)'][0] if '每年基本养老金购买额(实际)' in info[0] else np.nan)
            agent_trajectory['tda_pen_balance'].append(info[0]['个人养老金实际余额'][0])
            agent_trajectory['basic_pen_balance'].append(info[0]['基本养老金个人账户余额'][0] if '基本养老金个人账户余额' in info[0] else np.nan)
            agent_trajectory['tax'].append(info[0]['每年收入应缴税额(万元)'][0] if '每年收入应缴税额(万元)' in info[0] else np.nan)
            agent_trajectory['tda_tax_savings'].append(info[0]['个人养老金退税额'][0] if '个人养老金退税额' in info[0] else np.nan)
            agent_trajectory['basic_tax_savings'].append(info[0]['基本养老金退税额'][0] if '基本养老金退税额' in info[0] else np.nan)
            agent_trajectory['tda_tax'].append(info[0]['当期个人养老金缴税额'][0] if '当期个人养老金缴税额' in info[0] else np.nan)

        all_trajectories.append(agent_trajectory)
    pbar.close()
        
    # 计算所有agent的reward统计信息
    all_age = np.concatenate([traj['age'] for traj in all_trajectories])
    # import pandas as pd
    # age_income = pd.DataFrame({'age':all_age,'income':all_income})
    # mean_income_by_age = age_income.groupby('age')['income'].mean()
    action_set = [traj['actions'] for traj in all_trajectories]
    max_length = max(len(path) for path in action_set)
    unique_age = np.arange(env.get_attr('tb')[0], env.get_attr('td')[0] + 1)[:max_length]
    num_action = np.shape(action_set[0])[1]
    aligned_actions = np.full((max_length, num_action, eval_episodes), np.nan)
    for i, actions in enumerate(action_set):
        aligned_actions[:len(actions), :, i] = actions


    data = {'income_history_df':var_path(list([traj['raw_income'] for traj in all_trajectories]), eval_episodes, env),
            'gamma':env.get_attr('gamma')[0],
            'distf':env.get_attr('beta')[0],
            'period':env.get_attr('tn')[0],
            'tb':env.get_attr('tb')[0],
            'tr':env.get_attr('tr')[0],
            'reward_history_df':var_path(list([traj['rewards'] for traj in all_trajectories]), eval_episodes, env),
            'wealth_history_df':var_path(list([traj['cash'] for traj in all_trajectories]), eval_episodes, env),
            'tda_pen_balance_history_df':var_path(list([traj['tda_pen_balance'] for traj in all_trajectories]), eval_episodes, env),
            'basic_pen_balance_history_df':var_path(list([traj['basic_pen_balance'] for traj in all_trajectories]), eval_episodes, env),
          'aligned_actions':aligned_actions,
          'consum_history_df':var_path(list([traj['consumption'] for traj in all_trajectories]), eval_episodes, env),
          'tda_pen_amount_history_df':var_path(list([traj['tda_pen_amount'] for traj in all_trajectories]), eval_episodes, env),
          'basic_pen_amount_history_df':var_path(list([traj['basic_pen_amount'] for traj in all_trajectories]), eval_episodes, env),
          'tax_history_df':var_path(list([traj['tax'] for traj in all_trajectories]), eval_episodes, env),
          'tda_tax_savings_history_df':var_path(list([traj['tda_tax_savings'] for traj in all_trajectories]), eval_episodes, env),
          'basic_tax_savings_history_df':var_path(list([traj['basic_tax_savings'] for traj in all_trajectories]), eval_episodes, env),
          'tda_tax_history_df':var_path(list([traj['tda_tax'] for traj in all_trajectories]), eval_episodes, env),
          'unique_age':unique_age}
    # 保存pension_balance_history_df
    # model_identifier = 'rl_' + idx.split('-', 1)[1] 
    # os.makedirs(eval_path + model_identifier + '//', exist_ok=True)
    
    # pension_balance_history_df.to_excel(eval_path + model_identifier + '//' + model_identifier + "_pension_balance_history.xlsx",index=False)
    
    return data


if __name__ == '__main__':

    env_model_path_set = {'pensionfund-v5-rural':'models//rural//pensionfund-v5-rural_run13//',
                        'pensionfund-v5-nonrural':'models//nonrural//pensionfund-v5-nonrural_run79//'}

    # 创建进程池
    # pool = mp.Pool()
    
    # # 循环执行evaluate_model
    eval_episodes = 1000
    eval_seed =   3687851522

    distf = 0.95
    nonrural_data = evaluate_model_rl(env_model_path_set['pensionfund-v5-nonrural'],'pensionfund-v5-nonrural',eval_episodes=eval_episodes, eval_seed=eval_seed)
    baseline_score = nonrural_data['reward_history_df'].iloc[:, 1:].values
    income_path_df = nonrural_data['income_history_df'].iloc[:, 1:].values

    distf = nonrural_data['distf']
    gamma = nonrural_data['gamma']
    period = nonrural_data['period']
    tb = nonrural_data['tb']
    tr = nonrural_data['tr']

    # 计算Consumption-equivalent variations
    baseline_c = (-np.nanmean(baseline_score) / np.sum(np.ones(period)*(distf ** np.arange(period))))**(1/(1-gamma))
    

    penlimit_list = [0.6,1.2,2,3,5]
        # 定义收入数值分组的分组
    income_bins = np.array([
        [1, 2],    
        [2, 3],
        [3, 4],
        [4, 5],
        [5, 6],
        [6, 7],
        [7, 10],
        [10, 13],
        [13, 30]
    ])
    agent_mean_income_before_tr = nonrural_data['income_history_df'].iloc[:tr, 1:].values.mean(axis=0) # 退休前平均年收
    
    cev = pd.DataFrame(index=[f'{int(start)}-{int(end)}' for start, end in income_bins],columns=[f'penlimit_{penlimit}万' for penlimit in penlimit_list])
    idx_income = 0
    idx_penlimit = 0
    from tqdm import tqdm
    for penlimit in tqdm(penlimit_list):
        params = {'pension_limit_yuan':penlimit}
        score_history= evaluate_model_rl(env_model_path_set['pensionfund-v5-nonrural'],'pensionfund-v5-nonrural',eval_episodes=eval_episodes, eval_seed=eval_seed,params=params)['reward_history_df'].iloc[:, 1:].values
          
        for start_income, end_income in income_bins:
            agent_mask = (start_income <= agent_mean_income_before_tr) & (agent_mean_income_before_tr < end_income)
            mean_score = np.nanmean(score_history[:,agent_mask])
            pen01_c = (-mean_score / np.sum(np.ones(period)*(distf ** np.arange(period))))**(1/(1-gamma))

            # 计算baseline在
            baseline_mean_score = np.nanmean(baseline_score[:,agent_mask])
            baseline_c = (-baseline_mean_score / np.sum(np.ones(period)*(distf ** np.arange(period))))**(1/(1-gamma))

            cev.loc[f'{int(start_income)}-{int(end_income)}',f'penlimit_{penlimit}万'] = (baseline_c - pen01_c) *100 / pen01_c

    cev.reset_index(inplace=True)
    cev.rename(columns={'index':'收入区间(万元)'},inplace=True)

    # 两个消费列为multicolumn,两个风险资产列为multicolumn, 仅显示到小数点后3位
    table_tex = cev.to_latex(index=False, multicolumn=True, multicolumn_format='c', float_format='%.2f')
    print(table_tex)
    table_tex = table_tex.split(r'\midrule', 1)[1]  # 去除\midrule以前的内容
    header = r'''
	\begin{table}[htbp]
        \caption{设置个人养老金缴费上限的效用损失}
        \label{t7-penlimit_cev}
\centering
\makebox[\textwidth][c]{
\begin{threeparttable}
\begin{tabular}{C{2cm}*{5}{C{1.5cm}}}
        \toprule
        \multicolumn{1}{c}{\multirow{2}{*}{收入区间(万元)}} & \multicolumn{5}{c}{个人养老金缴费上限(万元)} \\ \cmidrule(r){2-6}
        \multicolumn{1}{c}{} & 0.6 & 1.2 & 2 & 3 & 5 \\ \midrule
    '''
    ender = r'''\begin{footnotesize}
					注：
		\end{footnotesize}
        \end{threeparttable}}
        \end{table}
        '''
    table_tex = header + table_tex.replace('\n', ' ').replace('\\\\', '\\\\ \n') + '\n' + ender # 加入自定义表头(header和table_tex之间不要有空行)
    # 将table_tex导出为latex文件
    with open('table/t7-penlimit_cev.tex', 'w') as f:
        f.write(table_tex)
    print(table_tex)
    
    





### CFPS数据处理

In [ ]:
import csv
import os
from tqdm import tqdm
import pandas as pd
import numpy as np

def taxation_2018(income):
    # 将收入从万元转换为元
    taxable_income = income - 60000 # 60000为免税额
    
    if taxable_income <= 0:
        return income
    
    # 根据税率表确定税率和速算扣除数
    if taxable_income <= 36000:
        tax_rate = 0.03
        deduction = 0
    elif taxable_income <= 144000:
        tax_rate = 0.10
        deduction = 2520
    elif taxable_income <= 300000:
        tax_rate = 0.20
        deduction = 16920
    elif taxable_income <= 420000:
        tax_rate = 0.25
        deduction = 31920
    elif taxable_income <= 660000:
        tax_rate = 0.30
        deduction = 52920
    elif taxable_income <= 960000:
        tax_rate = 0.35
        deduction = 85920
    else:
        tax_rate = 0.45
        deduction = 181920

    # 计算税款
    tax = (taxable_income * tax_rate - deduction)

    return income - tax

def taxation_2011(income):
    taxable_income = income - 42_000 
    if taxable_income <= 0:
        return income
    if taxable_income <= 18000:
        tax_rate = 0.03
        deduction = 0
    elif taxable_income <= 54000:
        tax_rate = 0.10
        deduction = 1260
    elif taxable_income <= 108000:
        tax_rate = 0.20
        deduction = 6660
    elif taxable_income <= 420000:
        tax_rate = 0.25
        deduction = 12_060
    elif taxable_income <= 660000:
        tax_rate = 0.30
        deduction = 27_060
    elif taxable_income <= 960000:
        tax_rate = 0.35
        deduction = 66_060
    else:
        tax_rate = 0.45
        deduction = 162_060

    tax = (taxable_income * tax_rate - deduction)
    return income - tax

def calculate_pretax_income(year, post_tax_income, insurance=0, precision=0.01, max_iter=1000):
    """
    根据税后收入反推税前收入（2011-2018年个税规则）
    
    参数：
    post_tax_income: 税后收入（元）
    insurance: 五险一金等专项扣除（元/年）
    precision: 计算精度（元）
    max_iter: 最大迭代次数
    
    返回：
    税前收入（保留两位小数）
    """
    if year > 2019:
        
        # 二分法查找范围
        low = post_tax_income
        high = post_tax_income * 2  # 初始上限设为税后收入两倍
        
        for _ in range(max_iter):
            mid = (low + high) / 2
            current_post_tax = taxation_2018(mid)
            
            if abs(current_post_tax - post_tax_income) < precision:
                return round(mid, 2)
            
            if current_post_tax < post_tax_income:
                low = mid
            else:
                high = mid
        
        return round((low + high)/2, 2)
    else:
        
        # 二分法查找范围
        low = post_tax_income
        high = post_tax_income * 2  # 初始上限设为税后收入两倍
        
        for _ in range(max_iter):
            mid = (low + high) / 2
            current_post_tax = taxation_2011(mid)
            
            if abs(current_post_tax - post_tax_income) < precision:
                return round(mid, 2)
            
            if current_post_tax < post_tax_income:
                low = mid
            else:
                high = mid
        
        return round((low + high)/2, 2)




'''
变量选择
pid: 个体唯一ID
age:年龄
qa301:户口类型
emp_income: 过去12个月所有工作（主要工作+一般工作）的税后工资性收入
retire: 是否退休 （“已退休”/“不适用”）
'''
var_list = ['pid','age','qa301','emp_income','retire']


whole_path = os.getcwd()
filename = {2022:'cfps2022person.csv',
            2020:'cfps2020person.csv',
            2018:'cfps2018person.csv',
            2016:'cfps2016person.csv',
            2014:'cfps2014person.csv',
            }

'''
处理城镇数据
'''



da_non_rural = pd.DataFrame()
for year, file in tqdm(filename.items()):
    df = pd.read_csv(whole_path + '/' + file, delimiter=",", encoding='gbk')
    df = df.loc[:,var_list]
    df['year'] = year

    df.loc[df['emp_income'].isin(['不适用','缺失']), 'emp_income'] = np.nan
    df.loc[:,'emp_income'] = df.loc[:,'emp_income'].astype(np.float64)
    df.loc[df['retire'].isin(['不适用']), 'retire'] = 0
    df.loc[df['retire'].isin(['已退休']), 'retire'] = 1
    df = df.dropna()
    df_1 = df.loc[df.loc[:,'qa301'].isin(['非农业户口', '居民户口', '农业户口']),:]
    df_non_rural = df_1.loc[df_1.loc[:,'qa301'].isin(['非农业户口', '居民户口']),:]
    df_non_rural['emp_income_before_tax'] = df_non_rural['emp_income'].apply(lambda x: calculate_pretax_income(year,x))

    '''
    保留未退休的数据
    '''
    df_non_rural_non_retire = df_non_rural.loc[df_non_rural.loc[:,'retire'] == 0,:]

    da_non_rural = pd.concat([da_non_rural, df_non_rural_non_retire], ignore_index=True)


# 保存为csv
da_non_rural.to_csv(whole_path + '/da_non_rural.csv', index=False,encoding='gbk')


'''
处理城乡数据

'''
da_rural = pd.DataFrame()
for year, file in tqdm(filename.items()):
    df = pd.read_csv(whole_path + '/' + file, delimiter=",", encoding='gbk')
    df = df.loc[:,var_list]
    df['year'] = year

    df.loc[df['emp_income'].isin(['不适用','缺失']), 'emp_income'] = np.nan
    df.loc[:,'emp_income'] = df.loc[:,'emp_income'].astype(np.float64)
    df.loc[df['retire'].isin(['不适用']), 'retire'] = 0
    df.loc[df['retire'].isin(['已退休']), 'retire'] = 1
    df = df.dropna()
    df_1 = df.loc[df.loc[:,'qa301'].isin(['非农业户口', '居民户口', '农业户口']),:]
    df_rural = df_1.loc[df_1.loc[:,'qa301'].isin(['农业户口']),:]

    df_rural['emp_income_before_tax'] = df_rural['emp_income'].apply(lambda x: calculate_pretax_income(year,x))

    '''
    保留未退休的数据
    '''
    df_rural_non_retire = df_rural.loc[df_rural.loc[:,'retire'] == 0,:]

    da_rural = pd.concat([da_rural, df_rural_non_retire], ignore_index=True)

# 保存为csv
da_rural.to_csv(whole_path + '/da_rural.csv', index=False,encoding='gbk')


### 表t5: 收入过程估计

In [ ]:
import csv
import os
from tqdm import tqdm
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from scipy import interpolate
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from numdifftools import Hessian

def estimate_income_process(residuals):
    # ================== 数据预处理 ==================
    # 按个体和时间排序
    df = residuals.sort_values(['pid', 'age']).copy()
    
    # 为每个个体生成时间序列结构
    individuals = []
    for pid, group in df.groupby('pid'):
        ages = group['age'].values
        e_it = group['e_it'].values
        deltas = np.diff(ages)  # 计算观测间隔
        individuals.append({
            'pid': pid,
            'ages': ages,
            'e_it': e_it,
            'deltas': deltas
        })

    # ================== 定义卡尔曼滤波似然函数 ==================
    def kalman_loglikelihood(params):
        rho, sigma_u, sigma_z = params
        sigma_u_sq = sigma_u**2
        sigma_z_sq = sigma_z**2
        
        total_loglik = 0.0
        
        for ind in individuals:
            # 初始化状态和协方差（使用稳态分布）
            if np.abs(rho) < 1:
                P = sigma_z_sq / (1 - rho**2)  # 稳态协方差
            else:
                P = 1e6  # 扩散初始化
            
            M = 0.0  # 初始状态期望
            loglik = 0.0
            
            ages = ind['ages']
            e_it = ind['e_it']
            deltas = ind['deltas']
            
            for i in range(len(e_it)):
                # 第一个观测不需要预测间隔
                if i > 0:
                    delta = deltas[i-1]
                    
                    # 计算转移系数和过程噪声
                    rho_d = rho**delta
                    Q = (sigma_z_sq * (1 - rho**(2*delta))) / (1 - rho**2) if rho != 1 else sigma_z_sq * delta
                    
                    # 预测步骤
                    M_pred = rho_d * M
                    P_pred = (rho_d**2) * P + Q
                else:
                    # 初始观测直接进入更新
                    M_pred = M
                    P_pred = P
                
                # 更新步骤
                y = e_it[i]
                resid = y - M_pred
                S = P_pred + sigma_u_sq  # 预测残差方差
                
                # 避免数值问题
                if S <= 0:
                    return np.inf
                
                # 计算对数似然
                loglik += -0.5 * (np.log(2*np.pi) + np.log(S) + (resid**2)/S)
                
                # 更新卡尔曼增益
                K = P_pred / S
                M = M_pred + K * resid
                P = (1 - K) * P_pred
            
            total_loglik += loglik
        
        # 返回负对数似然用于最小化
        return -total_loglik

# ======== 新增部分：计算标准误 ========
    def compute_standard_errors(result):
        # 计算Hessian矩阵
        hessian_func = Hessian(kalman_loglikelihood)
        H = hessian_func(result.x)
        
        # 处理可能的奇异矩阵
        try:
            cov_matrix = np.linalg.inv(H)
        except np.linalg.LinAlgError:
            # 添加正则化项处理非正定问题
            cov_matrix = np.linalg.pinv(H + 1e-6*np.eye(len(H)))
        
        # 计算标准误
        se = np.sqrt(np.diag(cov_matrix))
        return se
    
    # 参数估计
    initial_guess = [0.7, 0.2, 0.3]
    bounds = [
        (-0.99, 0.99),
        (1e-6, None),
        (1e-6, None)
    ]
    
    # 执行优化
    result = minimize(kalman_loglikelihood, initial_guess, 
                     method='L-BFGS-B', bounds=bounds,
                     options={'maxiter': 1000, 'ftol': 1e-8})

    if result.success:
        se = compute_standard_errors(result)
        params_est = {
            'rho': (result.x[0], se[0]),
            'sigma_u': (result.x[1], se[1]),
            'sigma_z': (result.x[2], se[2])
        }
    else:
        raise ValueError("Optimization failed: " + result.message)
    
    return params_est

'''
变量选择
pid: 个体唯一ID
age:年龄
qa301:户口类型
emp_income: 过去12个月所有工作（主要工作+一般工作）的税后工资性收入
retire: 是否退休 （“已退休”/“不适用”）
'''
var_list = ['pid','age','qa301','emp_income','retire']


'''
回归:
emp_income_before_tax = a + b * age + c * age^2 + d * age^3 + e * age^4 
'''

# 读取数据
da_rural = pd.read_csv('./cfps_data/da_rural.csv', delimiter=",", encoding='gbk')
da_non_rural = pd.read_csv('./cfps_data/da_non_rural.csv', delimiter=",", encoding='gbk')

np.sum(da_rural.groupby(by='pid').count()['age']<3)


from statsmodels.tools.tools import add_constant
from linearmodels.panel.model import PanelOLS
from linearmodels.iv.model import IVGMM
from arch import arch_model
import numpy as np


result_table = pd.DataFrame([])
for da in [da_non_rural,da_rural]:
    # ================== 数据预处理修正 ==================
    data = da.set_index(['pid', 'year'])
    data = data.dropna() 
    data = data.loc[data['emp_income_before_tax'] >= 10, :]
    data['age'] = data['age'].apply(lambda x: int(x) if x.isdigit() else np.nan)
    if da is da_rural:
        data = data.loc[(data['age'] >= 18) & (data['age'] <= 69), :]
    else:
        data = data.loc[(data['age'] >= 18) & (data['age'] <= 60), :]

    # 修正1：删除观测次数不足的个体（至少2个不同年龄观测）
    data = data.groupby('pid').filter(lambda x: x['age'].nunique() >= 2)

    # 修正2：线性插值替代样条插值（保持时间序列特性）


    data = data.reset_index(drop=False)[['pid','year','age','emp_income_before_tax']]
    data['income'] = data['emp_income_before_tax'] / 10000  # 转换为万元
    # data = data.groupby('pid').apply(spline_interpolate_individual_data).reset_index(drop=True)
    data['log_income'] = np.log(data['income'])
    data = data[['pid', 'year', 'age', 'log_income']].set_index(['pid', 'year'])
    data['age^2'] = data['age'] ** 2
    data['age^3'] = data['age'] ** 3
    data['age^4'] = data['age'] ** 4
    # ================== 收入趋势回归修正 ==================
    # 修正3：使用混合OLS而非面板固定效应
    from statsmodels.api import OLS, add_constant

    X = data[['age', 'age^2', 'age^3', 'age^4']].assign(const=1)
    y = data['log_income']
    model = OLS(y, X).fit()
    data['e_it'] = model.resid
    print(model.params.to_frame(name="Coefficient").values)
    # data['y_hat'] = model.predict(X)
    # np.dot(np.squeeze(model.params.to_frame(name="Coefficient").values),np.array([18,18**2,18**3,18**4,1]))
    # ================== 矩条件计算修正 ==================
    # 修正4：明确时间间隔（假设数据为年度）
    residuals = data.reset_index()[['pid', 'age', 'e_it']]

    '''
    cocco(2005):
    '''
    # # 生成delta_e_it
    # residuals_delta = pd.DataFrame()
    # # 循环每个pid,计算delta_e_it
    # for pid in residuals['pid'].unique():
    #     tmp = residuals.loc[residuals['pid'] == pid]
    #     age_list = sorted(tmp['age'].unique())  # 从小到大排序
    #     delta_e_it = pd.DataFrame()
    #     for i in range(len(age_list)-1):
    #         if len(tmp.loc[tmp['age'] == age_list[i]][['age','e_it']].values)>1:
    #             continue
    #         else:
    #             tmp_delta = tmp.loc[tmp['age'] > age_list[i]][['age','e_it']] - tmp.loc[tmp['age'] == age_list[i]][['age','e_it']].values
    #             tmp_delta.rename(columns={'e_it':'delta_e','age':'delta_age'},inplace=True)
    #             delta_e_it = pd.concat([delta_e_it, tmp_delta])

    #     residuals_delta = pd.concat([residuals_delta, delta_e_it])
    # # 删去大于10的delta_e_it
    # residuals_delta = residuals_delta.loc[residuals_delta['delta_age'] < 9,:]

    # # 
    # residuals_delta_var = residuals_delta.groupby('delta_age')['delta_e'].var().reset_index(name='delta_e_var')

    # # 回归
    # X = residuals_delta_var[['delta_age']].assign(const=1)
    # y = residuals_delta_var['delta_e_var']
    # model = OLS(y, X).fit()
    # # 获取参数
    # rho_est = model.params

    # print("估计参数:")
    # print(f"rho = 1")
    # print(f"sigma_u^2 = {(rho_est['const']/2):.4f}")
    # print(f"sigma_z^2 = {(rho_est['delta_age']):.4f}")

    '''
    卡尔曼滤波
    '''
    estimated_params = estimate_income_process(residuals)
    print("参数估计值（标准误）：")
    print(f"rho = {estimated_params['rho'][0]:.4f} ({estimated_params['rho'][1]:.4f})")
    print(f"sigma_u^2 = {estimated_params['sigma_u'][0]**2:.4f} ({estimated_params['sigma_u'][1]:.4f})")
    print(f"sigma_z^2 = {estimated_params['sigma_z'][0]**2:.4f} ({estimated_params['sigma_z'][1]:.4f})")
    


    # 提取结果
    sigma_u_sq_est, sigma_z_sq_est, rho_est = estimated_params['sigma_u'][0]**2, estimated_params['sigma_z'][0]**2, estimated_params['rho'][0]
    sigma_z_squared_se = estimated_params['sigma_z'][1]
    sigma_u_squared_se = estimated_params['sigma_u'][1]
    rho_est_se = estimated_params['rho'][1]


    '''
    记录面板结果
    '''
    panel_coef = model.params.to_frame(name="Coefficient").reset_index(drop=False).rename(columns={"index":'Variables'})


    panel_std = model.bse.to_frame(name="Coefficient")
    # 将panel_std插入到panel_coef中
    merge_coef = pd.DataFrame()
    for i in range(len(panel_coef)):
        merge_coef = pd.concat([merge_coef, panel_coef.iloc[[i]], panel_std.iloc[[i]]]).reset_index(drop=True) # 在每个coef的下一行插入空行


    income_result = pd.DataFrame({'Coefficient':[rho_est, rho_est_se, sigma_z_sq_est, sigma_z_squared_se, sigma_u_sq_est, sigma_u_squared_se, len(data), len(data.reset_index(drop=False)['pid'].unique())]},index=['rho',np.nan,f'sigma_z^2',np.nan,f'sigma_u^2',np.nan,'观测值','个体数'])
    
    merge_coef = pd.concat([merge_coef, 
                            income_result.reset_index(drop=False).rename(columns={"index":'Variables'})], axis=0)
    # 在最后一行增加观测值


    merge_coef.rename(columns={"Coefficient":'城乡' if da is da_rural else '城镇'},inplace=True)

    result_table = pd.concat([result_table, merge_coef.iloc[:,1]], axis=1)




result_table.insert(0, 'Variables', merge_coef.iloc[:,0])
# 将存在NaN的行的后几个对应数值都修改为(数字)
std = result_table.loc[result_table['Variables'].isna(), ['城乡','城镇']] 
result_table.loc[result_table['Variables'].isna(), ['城乡','城镇']]  = std.applymap(lambda x: '(' + '%.3f' % x + ')' if pd.notna(x) else x)

# 替换字符
result_table.replace('age', 'Age', inplace=True)
result_table.replace('age^2', 'Age$^2$', inplace=True)
result_table.replace('age^3', 'Age$^3$', inplace=True)
result_table.replace('age^4', 'Age$^4$', inplace=True)
result_table.replace('sigma_u^2', '$\\sigma^2_u$', inplace=True)
result_table.replace('sigma_z^2', '$\\sigma^2_z$', inplace=True)
result_table.replace('rho', '$\\rho$', inplace=True)
result_table.replace('const', '截距', inplace=True)

# 保存为tex
table_tex = result_table.to_latex(index=False, multicolumn=True, column_format='c' * len(result_table.columns), float_format='%.6f')


table_tex = table_tex.split(r'\midrule', 1)[1]  # 去除\midrule以前的内容
# 将表格中的NaN替换为'--'
table_tex = table_tex.replace('NaN', '')
header = r'''
\begin{table}[htbp]
\caption{收入过程估计结果}
\label{t5-income_estimation}
\centering
\begin{threeparttable}
\begin{tabular}{C{5cm}C{5cm}C{5cm}}
\toprule
Variables  & 城镇            & 城乡                \\ \midrule
'''
ender = r'''\begin{footnotesize}
        注:
    \end{footnotesize}
    \end{threeparttable}
    \end{table}
    '''

table_tex = header + table_tex.replace('\n', ' ').replace('\\\\', '\\\\ \n') + '\n' + ender # 加入自定义表头和表尾(header和table_tex之间不要有空行)
# 将table_tex导出为latex文件
with open('table/t5-income_estimation.tex', 'w',encoding='utf-8') as f:
    f.write(table_tex)
print(table_tex)






## 敏感性和异质性分析

### 税率的影响

### 养老金收益水平的影响

### 风险规避倾向

### 延迟退休

# 废弃

### 图p3：收入和持续收入冲击的10000条模拟路径

In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
from sbx import SAC
# from utils.sac import SAC
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
from tqdm import tqdm
# import matplotlib
# print(matplotlib.matplotlib_fname())
# raise

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)


# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')

# 将变量处理为终身路径
def var_path(vars, eval_episodes, env):
    max_length = max(len(path) for path in vars)
    var_path_array = np.full((eval_episodes, max_length), np.nan)
    for i, path in enumerate(vars):
        var_path_array[i, :len(path)] = path
    age_array = np.arange(env.get_attr('tb')[0], env.get_attr('tb')[0] + max_length)
    full_var_path = np.column_stack((age_array, var_path_array.T))
    var_path_df = pd.DataFrame(full_var_path,
                                    columns=['年龄'] + [f'个体{i+1}' for i in range(eval_episodes)])
    return var_path_df

import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄
register(
    id='pensionfund-v5',                                # call it whatever you want
    entry_point='v5_pensionfund:PensionFundEnv', # module_name:class_name
)


# model_name = [d for d in os.listdir(log_dir) if os.path.isdir(os.path.join(log_dir, d))]
# eval_episodes =201
eval_seed =   3687851522
distf = 0.95
import multiprocessing as mp
from functools import partial

def evaluate_model_naive(eval_episodes=10000, eval_seed=3687851522):
    # 读取cocco模型的A,C,V,gcash


    params = {'pension_limit':1}
    params = None

    env = make_vec_env("pensionfund-v5",seed=eval_seed, n_envs=1,  env_kwargs={'params': params})

    # 读取matlab dp解
    current_path = os.path.dirname(os.path.abspath(__file__))
    age_group = str(env.get_attr('tb')[0])+'-'+str(env.get_attr('td')[0])
    df_A = pd.read_excel(os.path.join(current_path,'result_cocco_matlab',age_group,'A.xlsx'),header=None)
    df_C = pd.read_excel(os.path.join(current_path,'result_cocco_matlab',age_group,'C.xlsx'),header=None) 
    df_V = pd.read_excel(os.path.join(current_path,'result_cocco_matlab',age_group,'V.xlsx'),header=None)  
    df_gcash = pd.read_excel(os.path.join(current_path,'result_cocco_matlab',age_group,'gcash.xlsx'),header=None)
    # 转换为numpy数组
    A = df_A.to_numpy()
    C = df_C.to_numpy()
    V = df_V.to_numpy()
    gcash = df_gcash.to_numpy().squeeze()



    # env = make_vec_env("cocco-dp",seed=1, n_envs=1,  env_kwargs={'params': None})

    action_set = []
    score_history = []
    age = []
    income = []
    consum = []
    tmp_income = []
    temp_consum = []
    persist_shock = []
    tmp_persist_shock = []

    reward_history = []

    # from tqdm import tqdm
    pbar = tqdm(total=eval_episodes, desc=f'dp模拟进度')
    for episode in range(eval_episodes):     
        pbar.update(1)
        env.seed(seed=eval_seed+episode)
        obs = env.reset()   
        info = env.reset_infos
        done = False
        score = 0
        episode_actions = []
        temp_reward = []
        while not done:
            # 根据state[0]获取现金
            normalized_cash = np.squeeze(info[0]['normalized_cash']) # 归一化的现金
            cash = obs[0][0]  # 现金
            age_loc =  env.get_attr('age')[0] - env.get_attr('tb')[0] # 获取年龄

            # 根据现金和年龄插值从A中获取action
            action = np.array([np.interp(normalized_cash, gcash, C[:, age_loc]),
                               np.interp(normalized_cash, gcash, A[:, age_loc]),0,0])
            if age_loc == np.shape(C)[1]-1: # 如果到达最后一期,则固定消费比例为1（全部消费干净）
                action[0] = normalized_cash
            else:
                action[0] = np.clip(action[0],0,normalized_cash)
            # action[0] = action[0]/cash # 转换为消费比例
            action[0] = action[0]/(normalized_cash) # 消费比例  

            episode_actions.append(action)
            # action[0] = 1 #          
            action[1] = np.clip(action[1],0,1) # 风险资产比例
            obs, reward, done, info = env.step(np.array([action]))
            score += reward * (distf ** len(episode_actions))
            tmp_income.append(info[0]['basic_income'])
            if info[0]['status']=='working':               
                tmp_persist_shock.append(obs[0][1])
            else:
                tmp_persist_shock.append(np.nan)
        persist_shock.append(tmp_persist_shock)
        action_set.append(episode_actions)
        age.append(info[0]['age'][0])
        income.append(tmp_income)
        tmp_income = []
        tmp_persist_shock = []

    max_length = max(len(path) for path in action_set)
    unique_age = np.arange(env.get_attr('tb')[0], env.get_attr('td')[0] + 1)[:max_length]

    income_history_df = var_path(income, eval_episodes, env)
    persist_shock_history_df = var_path(persist_shock, eval_episodes, env)
    return unique_age, income_history_df, persist_shock_history_df

if __name__ == '__main__':   
    tr=46
    unique_age, income_history_df, persist_shock_history_df = evaluate_model_naive()
    income_history_df = income_history_df.iloc[:,1:]
    persist_shock_history_df = persist_shock_history_df.iloc[:tr,1:]

    
    
    import matplotlib.pyplot as plt

    # 设置字体为SimHei以显示中文，若无法显示中文，请确保系统中已安装该字体
    plt.rcParams['font.sans-serif'] = ['SimHei']  
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

    # 左图画收入，右图画持续冲击
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(9, 4))
    
    income_history_df_sample = income_history_df.sample(200,axis=1)
    persist_shock_history_df_sample = persist_shock_history_df.sample(200,axis=1)

    # 左图：画出所有收入路径，平均收入路径加粗，其余路径很浅的颜色
    # 随机挑选200条路径
    
    for i in range(200):  # 选取200条路径
        ax1.plot(unique_age, income_history_df_sample.iloc[:, i], alpha=0.2, linewidth=1, color='gray')
    # 选取其中一条虚线设置label 
    ax1.plot(unique_age, income_history_df_sample.iloc[:, 0], alpha=0.2, linewidth=1, color='gray', label='模拟家庭收入')
    ax1.plot(unique_age, income_history_df.mean(axis=1), linewidth=2.5, color='blue', label='平均收入') # 平均值
    ax1.plot(unique_age, income_history_df.median(axis=1), linewidth=2.5, color='red', label='中位数收入') # 中位数

    # 在退休年龄处画一条垂直于x轴的无色线，并填充右侧的空间
    ax1.axvline(x=tr+20, color='gray', linestyle='--', linewidth=0.5)
    ax1.fill_betweenx(ax1.get_ylim(), tr+20, unique_age[-1], color='gray', alpha=0.1)

    # 获取收入和永久收入冲击的最大值和最小值
    income_min, income_max = income_history_df_sample.min().min(), income_history_df_sample .max().max()
    persist_shock_min, persist_shock_max = persist_shock_history_df_sample.min().min(), persist_shock_history_df_sample.max().max()

    # 设置y轴范围
    ax1.set_ylim(income_min, income_max)
    ax2.set_ylim(persist_shock_min, persist_shock_max)

    # 获取年龄的最小值和最大值
    age_min, age_max = unique_age.min(), unique_age.max()

    # 在右侧加一个文本
    ax1.text(tr+35, ax1.get_ylim()[1]-2, '退休期', ha='center', va='bottom', fontsize=12,color='gray')

    # 设置x轴范围
    ax1.set_xlim(age_min, age_max)
    ax2.set_xlim(age_min, tr+20)
    ax1.legend(loc='upper left')
    ax1.set_xlabel('年龄', fontsize=12)
    ax1.set_ylabel('收入(万元)', fontsize=12)
    ax1.grid(True, linestyle='--', alpha=1)

    # 右图：画出每个工作期永久收入冲击路径，平均永久收入冲击路径加粗，其余路径很浅的颜色
    
    for i in range(200): # 选取200条路径
        ax2.plot(unique_age[:tr], persist_shock_history_df_sample.iloc[:, i], alpha=0.2, linewidth=0.7, color='gray')
    ax2.plot(unique_age[:tr], persist_shock_history_df_sample.iloc[:, 0], alpha=0.2, linewidth=0.7, color='gray', label='模拟家庭持续收入冲击')
    ax2.plot(unique_age[:tr], persist_shock_history_df.mean(axis=1), linewidth=2.5, color='blue', label='平均持续收入冲击') # 平均值

    ax2.legend(loc='upper left')
    ax2.set_xlabel('年龄', fontsize=12)
    ax2.set_ylabel('持续收入冲击', fontsize=12)
    ax2.grid(True, linestyle='--', alpha=1)

    # 设置子图的标题
    ax1.set_title('(a) $Y_t$', fontsize=14)
    ax2.set_title('(b) $M_t$', fontsize=14)

    plt.tight_layout()
    plt.savefig('figs/p3-income_and_persist_shock_path.png', dpi=300)
    plt.show()




### 图p4：DP与RL求解结果比较

In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
from sbx import SAC
# from utils.sac import SAC
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)


# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')

# 将变量处理为终身路径
def var_path(vars, eval_episodes, env):
    max_length = max(len(path) for path in vars)
    var_path_array = np.full((eval_episodes, max_length), np.nan)
    for i, path in enumerate(vars):
        var_path_array[i, :len(path)] = path
    age_array = np.arange(env.get_attr('tb')[0], env.get_attr('tb')[0] + max_length)
    full_var_path = np.column_stack((age_array, var_path_array.T))
    var_path_df = pd.DataFrame(full_var_path,
                                    columns=['年龄'] + [f'个体{i+1}' for i in range(eval_episodes)])
    return var_path_df

import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄
register(
    id='cocco-v0',                                # call it whatever you want
    entry_point='v0_cocco:CoccoEnv', # module_name:class_name
)

log_dir = 'models//sac'

eval_dir = 'eval//'
# model_name = ['v13_pensionfund_penlim0_run7',
#               'v13_pensionfund_penlim1p2_run15',
#               'v13_pensionfund_nopenlim_run7'
#               ]
rl_model_name = 'cocco-v0_run14//RT//RT_4//best_model' # 
rl_vec_env_name = 'cocco-v0_run14//RT//RT_4//best_model_vec_env.pkl'


# model_name = [d for d in os.listdir(log_dir) if os.path.isdir(os.path.join(log_dir, d))]
eval_episodes = 200
eval_seed =   3687851522
distf = 0.95
import multiprocessing as mp
from functools import partial

def evaluate_model_naive(eval_episodes=200, eval_seed=3687851522):
    # 读取cocco模型的A,C,V,gcash


    params = {'pension_limit':1}
    params = None

    env = make_vec_env("cocco-v0",seed=eval_seed, n_envs=1,  env_kwargs={'params': params})

    # 读取matlab dp解
    current_path = os.path.dirname(os.path.abspath(__file__))
    age_group = str(env.get_attr('tb')[0])+'-'+str(env.get_attr('td')[0])
    df_A = pd.read_excel(os.path.join(current_path,'result_cocco_matlab/cocco/A.xlsx'),header=None)
    df_C = pd.read_excel(os.path.join(current_path,'result_cocco_matlab/cocco/C.xlsx'),header=None) 
    df_V = pd.read_excel(os.path.join(current_path,'result_cocco_matlab/cocco/V.xlsx'),header=None)  
    df_gcash = pd.read_excel(os.path.join(current_path,'result_cocco_matlab/cocco/gcash.xlsx'),header=None)
    # 转换为numpy数组
    A = df_A.to_numpy()
    C = df_C.to_numpy()
    V = df_V.to_numpy()
    gcash = df_gcash.to_numpy().squeeze()



    # env = make_vec_env("cocco-dp",seed=1, n_envs=1,  env_kwargs={'params': None})

    action_set = []
    score_history = []
    age = []
    income = []
    consum = []
    tmp_income = []
    temp_consum = []
    cash_path = []
    temp_cash = []
    
    reward_history = []

    # from tqdm import tqdm
    pbar = tqdm(total=eval_episodes, desc=f'dp模拟进度')
    for episode in range(eval_episodes):     
        pbar.update(1)
        env.seed(seed=eval_seed+episode)
        obs = env.reset()   
        info = env.reset_infos
        done = False
        score = 0
        episode_actions = []
        temp_reward = []
        while not done:
            # 根据state[0]获取现金
            normalized_cash = np.squeeze(info[0]['normalized_cash']) # 归一化的现金
            cash = obs[0][0]  # 现金
            age_loc =  env.get_attr('age')[0] - env.get_attr('tb')[0] # 获取年龄

            # 根据现金和年龄插值从A中获取action
            action = np.array([np.interp(normalized_cash, gcash, C[:, age_loc]),
                               np.interp(normalized_cash, gcash, A[:, age_loc])])
            if age_loc == np.shape(C)[1]-1: # 如果到达最后一期,则固定消费比例为1（全部消费干净）
                action[0] = normalized_cash
            else:
                action[0] = np.clip(action[0],0,normalized_cash)
            # action[0] = action[0]/cash # 转换为消费比例
            action[0] = action[0]/(normalized_cash) # 消费比例  
            # action[0] = 1 #          
            action[1] = np.clip(action[1],0,1) # 风险资产比例



            obs, reward, done, info = env.step(np.array([action]))
            score += reward * (distf ** len(episode_actions))

            episode_actions.append(action)
            temp_consum.append(info[0]['实际消费'])
            temp_reward.append(reward[0])
            temp_cash.append(info[0]['real_cash'])
            # if info[0]['status']=='working':

        age.append(info[0]['age'][0])
        score_history.append(score[0])
        action_set.append(episode_actions)
        reward_history.append(temp_reward)
        consum.append(temp_consum)
        cash_path.append(temp_cash)
        temp_consum = []
        temp_cash = []

    # 保存评估文件
    # 从模型名中提取标识符（第二个_之后的字符串）
    model_identifier = 'naive'

    os.makedirs(eval_dir + model_identifier + '//', exist_ok=True)


    life_summary = np.array([
        np.mean(age),
        np.mean(score_history),
        np.std(score_history),
    ])

    # =====保存现金路径=====
    cash_path_df = var_path(cash_path, eval_episodes, env)
    cash_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_cash_history.xlsx", index=False)

     # =====保存消费路径=====
    consum_path_df = var_path(consum, eval_episodes, env)
    consum_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_consum_history.xlsx", index=False)
 
    # 保存action
    max_length = max(len(path) for path in consum)
    unique_age = np.arange(env.get_attr('tb')[0], env.get_attr('td')[0] + 1)[:max_length]
    max_length = max(len(arr) for arr in action_set)
    num_action = np.shape(action_set[0])[1]
    aligned_actions = np.full((max_length, num_action, eval_episodes), np.nan)
    for i, actions in enumerate(action_set):
        aligned_actions[:len(actions), :, i] = actions

    mean_action_by_age = np.nanmean(aligned_actions, axis=2)
    mean_actions = np.column_stack((unique_age, mean_action_by_age))
    mean_actions_df = pd.DataFrame(mean_actions, 
                                    columns=['年龄', '消费', '风险资产'])
    mean_actions_path = eval_dir+ model_identifier + "//" + model_identifier + "_mean_actions.xlsx"
    mean_actions_df.to_excel(mean_actions_path, index=False)

    # 保存



    life_summary_df = pd.DataFrame([life_summary], 
                                columns=['死亡年龄', '终身效用','终身效用标准差'])
    # 在life_summary_df前面插入模型标识符列
    life_summary_df.insert(0, '模型', model_identifier)
    return aligned_actions, cash_path_df, consum_path_df



def evaluate_model_rl(model_name,vec_name, eval_episodes=1000, eval_seed=3687851522):
    current_path = os.path.dirname(os.path.abspath(__file__))    

    env = make_vec_env("cocco-v0",seed=1, n_envs=1, env_kwargs={'params': None},monitor_kwargs={'discount_factor':distf})
    

    env = VecNormalize.load(os.path.join(current_path, log_dir,vec_name), env)
    # env.obs_rms.mean
    # env.obs_rms.var
    
    model = SAC.load(os.path.join(current_path, log_dir, model_name), env=env)

    

    # env = make_vec_env("pensionfund-v1",seed=eval_seed, n_envs=1, env_kwargs={'params': None})
    
    # env = VecNormalize.load(os.path.join(current_path, log_dir, idx) + "//eval_env.pkl",env)

    env = make_vec_env("cocco-v0",seed=eval_seed, n_envs=1, env_kwargs={'params':None},monitor_kwargs={'discount_factor': distf})         
    env = VecNormalize(env, norm_obs=False, norm_reward=False) # 评估不需要norm_obs


    idx = model_name.split('//')[0]
    # model = DDPG.load(os.path.join(log_dir, idx) + "//latest_model", env)

    action_set = []
    score_history = []
    age = []
    consum = []
    temp_consum = []
    reward_history = []
    cash_path = []
    temp_cash = []
    # from tqdm import tqdm
    pbar = tqdm(total=eval_episodes, desc=f'模拟进度 {idx}')
    
    for episode in range(eval_episodes):       
        pbar.update(1)
        env.seed(seed=eval_seed+episode)
        obs = env.reset()    
        done = False
        score = 0
        episode_actions = []
        temp_reward = []
        while not done:
            action, _ = model.predict(model.env.normalize_obs(obs), deterministic=True) 
            # action[0,2] = 0
            cash = obs[0][0] #

            obs, reward, done, info = env.step(action)               
            # reward = -(((cash*action[0][0]))**(1-env.get_attr('gamma')[0]))*10 + 10       
            score += reward * (distf ** len(episode_actions))
            episode_actions.append(info[0]['real_actions'])
            temp_consum.append(info[0]['实际消费'])
            temp_reward.append(reward[0])
            temp_cash.append(info[0]['real_cash'])
        
        score_history.append(score[0])
        reward_history.append(temp_reward)
        action_set.append(episode_actions)
        age.append(info[0]['age'][0])
        consum.append(temp_consum)
        cash_path.append(temp_cash)
        temp_consum = []
        temp_cash = []

    # 保存评估文件
    # 从模型名中提取标识符（第二个_之后的字符串）
    model_identifier = 'rl_' + idx.split('-', 1)[1] 

    os.makedirs(eval_dir + model_identifier + '//', exist_ok=True)
    
        # =====保存现金路径=====
    cash_path_df = var_path(cash_path, eval_episodes, env)
    cash_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_cash_history.xlsx", index=False)



    life_summary = np.array([
        np.mean(age),
        np.mean(score_history),
        np.std(score_history),
    ])

    life_summary_df = pd.DataFrame([life_summary], 
                                columns=[ '死亡年龄', '终身效用','终身效用标准差'])

    # =====保存消费路径=====
    consum_path_df = var_path(consum, eval_episodes, env)
    consum_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_consum_history.xlsx", index=False)


    
       # =====保存奖励历史=====
    reward_history_df = var_path(reward_history, eval_episodes, env)
    reward_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_reward_history.xlsx", index=False)

    life_summary = np.array([
        np.mean(age),
        np.mean(score_history),
        np.std(score_history),
    ])

    life_summary_path = eval_dir+ model_identifier + "//" + model_identifier + "_life_summary.xlsx"
    life_summary_df.to_excel(life_summary_path, index=False)

    max_length = max(len(path) for path in consum)
    unique_age = np.arange(env.get_attr('tb')[0], env.get_attr('td')[0] + 1)[:max_length]
    max_length = max(len(arr) for arr in action_set)
    num_action = np.shape(action_set[0])[1]
    aligned_actions = np.full((max_length, num_action, eval_episodes), np.nan)
    for i, actions in enumerate(action_set):
        aligned_actions[:len(actions), :, i] = actions

    mean_action_by_age = np.nanmean(aligned_actions, axis=2)
    mean_actions = np.column_stack((unique_age, mean_action_by_age))
    mean_actions_df = pd.DataFrame(mean_actions, 
                                    columns=['年龄', '消费', '风险资产'])
    mean_actions_path = eval_dir+ model_identifier + "//" + model_identifier + "_mean_actions.xlsx"
    mean_actions_df.to_excel(mean_actions_path, index=False)
    
    #
    
    # 在life_summary_df前面插入模型标识符列
    life_summary_df.insert(0, '模型', model_identifier)
    return aligned_actions, cash_path_df, consum_path_df
    # print(f"模型{idx}评估完毕, 评估次数为{eval_episodes}")
# def evalute_model_erl(idx, eval_episodes=1000, eval_seed=3687851522):

if __name__ == '__main__':   
    tr=46
    aligned_actions_naive, cash_path_df_naive, consum_path_df_naive = evaluate_model_naive()
    aligned_actions_rl, cash_path_df_rl, consum_path_df_rl = evaluate_model_rl(rl_model_name,rl_vec_env_name,eval_episodes=eval_episodes, eval_seed=eval_seed)

    # 
    unique_age = cash_path_df_naive['年龄']
    consum_dp = consum_path_df_naive.iloc[:,1:].values
    consum_rl = consum_path_df_rl.iloc[:,1:].values
    # consum_pct_dp = aligned_actions_naive[:,0,:]
    # consum_pct_rl = aligned_actions_rl[:,0,:]
    risky_pct_dp = aligned_actions_naive[:,1,:]
    risky_pct_rl = aligned_actions_rl[:,1,:]

    cash_dp = cash_path_df_naive.iloc[:,1:].values
    cash_rl = cash_path_df_rl.iloc[:,1:].values

    # 设置字体路径
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用简体中文字体
    plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

    # 创建三个子图，分别用于consum、risky_pct和cash的对比
    fig, axs = plt.subplots(1, 3, figsize=(9, 3))  # （宽度，高度）

    # 左图：consum的dp和rl对比
    consum_dp_mean = np.nanmean(consum_dp,axis=1)
    consum_dp_std = np.nanstd(consum_dp,axis=1)
    consum_rl_mean = np.nanmean(consum_rl,axis=1)
    consum_rl_std = np.nanstd(consum_rl,axis=1)

    axs[0].fill_between(unique_age, 
                        consum_dp_mean - consum_dp_std, 
                        consum_dp_mean + consum_dp_std, 
                        color='blue', alpha=0.1)
    axs[0].fill_between(unique_age,
                        consum_rl_mean - consum_rl_std, 
                        consum_rl_mean + consum_rl_std, 
                        color='orange', alpha=0.2)
    axs[0].plot(unique_age, consum_dp_mean, color='blue', label='DP')
    axs[0].plot(unique_age, consum_rl_mean, color='orange', label='RL')
    axs[0].legend()
    axs[0].set_title('(a) 消费路径')
    axs[0].set_xlabel('年龄')
    axs[0].set_ylabel('消费额')
    axs[0].grid(True, which='both', linestyle='--', linewidth=0.5)  # 添加次要网格

    # 中图：risky_pct的dp和rl对比
    risky_dp_mean = np.nanmean(risky_pct_dp,axis=1)
    risky_dp_std = np.nanstd(risky_pct_dp,axis=1)
    risky_rl_mean = np.nanmean(risky_pct_rl,axis=1)
    risky_rl_std = np.nanstd(risky_pct_rl,axis=1)

    axs[1].fill_between(unique_age, 
                        risky_dp_mean - risky_dp_std, 
                        risky_dp_mean + risky_dp_std, 
                        color='blue', alpha=0.1)
    axs[1].fill_between(unique_age,
                        risky_rl_mean - risky_rl_std, 
                        risky_rl_mean + risky_rl_std, 
                        color='orange', alpha=0.2)
    axs[1].plot(unique_age, risky_dp_mean, color='blue', label='DP')
    axs[1].plot(unique_age, risky_rl_mean, color='orange', label='RL')
    axs[1].legend()
    axs[1].set_title('(b) 风险资产路径')
    axs[1].set_xlabel('年龄')
    axs[1].set_ylabel('风险资产比例')
    axs[1].grid(True, which='both', linestyle='--', linewidth=0.5)  # 添加次要网格

    # 右图：cash的dp和rl对比
    cash_dp_mean = np.nanmean(cash_dp,axis=1)
    cash_dp_std = np.nanstd(cash_dp,axis=1)
    cash_rl_mean = np.nanmean(cash_rl,axis=1)
    cash_rl_std = np.nanstd(cash_rl,axis=1)

    axs[2].fill_between(unique_age, 
                        cash_dp_mean - cash_dp_std, 
                        cash_dp_mean + cash_dp_std, 
                        color='blue', alpha=0.1)
    axs[2].fill_between(unique_age,
                        cash_rl_mean - cash_rl_std, 
                        cash_rl_mean + cash_rl_std, 
                        color='orange', alpha=0.2)
    axs[2].plot(unique_age, cash_dp_mean, color='blue', label='DP')
    axs[2].plot(unique_age, cash_rl_mean, color='orange', label='RL')
    axs[2].legend()
    axs[2].set_title('(c) 现金路径')
    axs[2].set_xlabel('年龄')
    axs[2].set_ylabel('现金')
    axs[2].grid(True, which='both', linestyle='--', linewidth=0.5)  # 添加次要网格

    

    plt.tight_layout()
    plt.savefig('./figs/p4-cocco_comparison.png',dpi=500)




### 表t2：不同收入区间的纳税和购买个人养老金节税结果

In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
from sbx import SAC
# from utils.sac import SAC
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
# import matplotlib
# print(matplotlib.matplotlib_fname())
# raise

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)


# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')

# 将变量处理为终身路径
def var_path(vars, eval_episodes, env):
    max_length = max(len(path) for path in vars)
    var_path_array = np.full((eval_episodes, max_length), np.nan)
    for i, path in enumerate(vars):
        var_path_array[i, :len(path)] = path
    age_array = np.arange(env.get_attr('tb')[0], env.get_attr('tb')[0] + max_length)
    full_var_path = np.column_stack((age_array, var_path_array.T))
    var_path_df = pd.DataFrame(full_var_path,
                                    columns=['年龄'] + [f'个体{i+1}' for i in range(eval_episodes)])
    return var_path_df

import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄
register(
    id='pensionfund-v5',                                # call it whatever you want
    entry_point='v5_pensionfund:PensionFundEnv', # module_name:class_name
)
log_dir = 'models//sac'

eval_dir = 'eval//'
# model_name = ['v13_pensionfund_penlim0_run7',
#               'v13_pensionfund_penlim1p2_run15',
#               'v13_pensionfund_nopenlim_run7'
#               ]
rl_model_name = 'pensionfund-v5_run33//RT//RT_9//RT//RT_5//best_model' # 
rl_vec_env_name = 'pensionfund-v5_run33//RT//RT_9//RT//RT_5//best_model_vec_env.pkl'


# model_name = [d for d in os.listdir(log_dir) if os.path.isdir(os.path.join(log_dir, d))]
eval_episodes = 10000
eval_seed =   3687851522
distf = 0.95
import multiprocessing as mp
from functools import partial

# def evalute_model_erl(idx, eval_episodes=1000, eval_seed=3687851522):


def evaluate_model_rl(model_name,vec_name, eval_episodes=1000, eval_seed=3687851522):
    current_path = os.path.dirname(os.path.abspath(__file__))    
    # try:
    with open(os.path.join(current_path, log_dir, model_name.split('//')[0]) +"//params.json", 'r') as file:
        params = json.load(file)   
    # except:
    params = {'pension_limit':1}
    # if 'cocco' in idx:
    #     env = make_vec_env("cocco-rl-norm",seed=1, n_envs=1, env_kwargs={'params': params})
    # else:
    # env = make_vec_env("pensionfund-v3",seed=1, n_envs=1, env_kwargs={'params': params})````````
    env = make_vec_env("pensionfund-v5",seed=1, n_envs=1, env_kwargs={'params': params},\
                            monitor_kwargs={'discount_factor':distf})
    

    env = VecNormalize.load(os.path.join(current_path, log_dir,vec_name), env)
    # env.obs_rms.mean
    # env.obs_rms.var
    
    model = SAC.load(os.path.join(current_path, log_dir, model_name), env=env)

    

    # env = make_vec_env("pensionfund-v1",seed=eval_seed, n_envs=1, env_kwargs={'params': None})
    
    # env = VecNormalize.load(os.path.join(current_path, log_dir, idx) + "//eval_env.pkl",env)

    env = make_vec_env("pensionfund-v5",seed=eval_seed, n_envs=1, env_kwargs={'params':params},\
                            monitor_kwargs={'discount_factor': distf})         
    env = VecNormalize(env, norm_obs=False, norm_reward=False) # 评估不需要norm_obs


    idx = model_name.split('//')[0]
    # model = DDPG.load(os.path.join(log_dir, idx) + "//latest_model", env)

    action_set = []
    age = []
    income = []
    tmp_income = []
    tax_saving = []
    tmp_tax_saving = []

    tax = []
    tmp_tax = []

    pension_tax = []
    tmp_pension_tax = []

    pen_amount = []
    tmp_pen_amount = []

    
    # from tqdm import tqdm
    pbar = tqdm(total=eval_episodes, desc=f'模拟进度 {idx}')
    
    for episode in range(eval_episodes):       
        pbar.update(1)
        env.seed(seed=eval_seed+episode)
        obs = env.reset()    
        done = False
        score = 0
        episode_actions = []
        while not done:
            action, _ = model.predict(model.env.normalize_obs(obs), deterministic=True) 
            # action[0,2] = 0

            obs, reward, done, info = env.step(action)               
            # reward = -(((cash*action[0][0]))**(1-env.get_attr('gamma')[0]))*10 + 10       
            score += reward * (distf ** len(episode_actions))
            episode_actions.append(info[0]['real_actions'])
            tmp_income.append(info[0]['basic_income'])
            # if info[0]['status']=='working':
            try:
                tmp_pen_amount.append(info[0]['每年个人养老金购买额(实际)'][0])
            except:
                tmp_pen_amount.append(np.nan)
            try:
                tmp_tax_saving.append(info[0]['每年个人养老金退税额'][0])
            except:
                tmp_tax_saving.append(np.nan)
            try:
                tmp_tax.append(info[0]['每年收入应缴税额(万元)'][0])
            except:
                tmp_tax.append(np.nan)
            try:
                tmp_pension_tax.append(info[0]['当期养老金缴税额'][0])
            except:
                tmp_pension_tax.append(np.nan)

        action_set.append(episode_actions)
        age.append(info[0]['age'][0])
        income.append(tmp_income)
        tax_saving.append(tmp_tax_saving)
        tax.append(tmp_tax)
        pension_tax.append(tmp_pension_tax)
        pen_amount.append(tmp_pen_amount)
        tmp_income = []
        tmp_tax_saving = []
        tmp_tax = []
        tmp_pension_tax = []
        tmp_pen_amount = []
    income_history_df = var_path(income, eval_episodes, env)
    tax_saving_df = var_path(tax_saving, eval_episodes, env)
    tax_df = var_path(tax, eval_episodes, env)
    pension_tax_df = var_path(pension_tax, eval_episodes, env)
    pen_amount_df = var_path(pen_amount, eval_episodes, env)


    max_length = max(len(path) for path in action_set)
    unique_age = np.arange(env.get_attr('tb')[0], env.get_attr('td')[0] + 1)[:max_length]
    num_action = np.shape(action_set[0])[1]
    aligned_actions = np.full((max_length, num_action, eval_episodes), np.nan)
    for i, actions in enumerate(action_set):
        aligned_actions[:len(actions), :, i] = actions
        

    return unique_age, aligned_actions, income_history_df, tax_saving_df, tax_df, pension_tax_df, pen_amount_df

if __name__ == '__main__':


    # 创建进程池
    # pool = mp.Pool()
    
    # # 循环执行evaluate_model
    tr = 46 # 退休年龄
    from tqdm import tqdm
    unique_age, rl_actions, income_history_df, tax_saving_df, tax_df, pension_tax_df, pen_amount_df = evaluate_model_rl(rl_model_name,rl_vec_env_name,eval_episodes=eval_episodes, eval_seed=eval_seed)
    # efficient_tax_rate = pd.concat([income_history_df.iloc[:, 0], (tax_df.iloc[:, 1:] / income_history_df.iloc[:, 1:])*100], axis=1) # 实际税率

    # 将数值转变为折现数值
    distf = 0.95
    tax_df.iloc[:, 1:] = tax_df.iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    tax_saving_df.iloc[:, 1:] = tax_saving_df.iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    pension_tax_df.iloc[:, 1:] = pension_tax_df.iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))
    pen_amount_df.iloc[:, 1:] = pen_amount_df.iloc[:, 1:].apply(lambda x: x * distf ** np.arange(len(x)))

    # 定义收入数值分组和年龄段的分组
    income_bins = np.array([
        [0, 3],
        [3+1e-06, 6],    
        [6+1e-06, 8],
        [8+1e-06, 10],
        [10+1e-06, 12],
        [12+1e-06, 14],
        [14+1e-06, 16],
        [16+1e-06, 18],
        [18+1e-06, 20],
        [20+1e-06, 100],
    ])
    age_bins = np.array([
        [20, 35],
        [36, 50],
        [51, 65],
    ])
    '''
    (1) 计算每个生涯平均收入分组的终身缴税额现值、收入现值和实际税率
    '''
    temp_income_before_tr = income_history_df.iloc[:tr, 1:].values # 退休前收入
    temp_dist_income_before_tr = income_history_df.iloc[:tr, 1:].apply(lambda x: x * distf ** np.arange(len(x))).values # 退休前折现收入
    agent_dist_sum_income_before_tr = np.nansum(temp_dist_income_before_tr,axis=0) # 退休前折现总收入
    agent_meanincome_before_tr = temp_income_before_tr.mean(axis=0) # 退休前平均年收

    # 税收
    eff_tax = tax_df.iloc[:tr, 1:].values # 工作期税收
    eff_tax_dist = tax_df.iloc[:tr, 1:].apply(lambda x: x * distf ** np.arange(len(x))).values # 工作期折现税收
    agent_dist_sum_tax_before_tr = np.nansum(eff_tax_dist,axis=0) # 工作期折现总

    # 循环
    tax_by_income = np.full((len(income_bins),3), np.nan) 
    idx_income = 0
    for start_income, end_income in income_bins: 
        agent_mask = (start_income <= agent_meanincome_before_tr) & (agent_meanincome_before_tr < end_income)
        tax_by_income[idx_income,0] = np.nanmean(agent_dist_sum_tax_before_tr[agent_mask]/agent_dist_sum_income_before_tr[agent_mask])*100 # 平均有效税率
        tax_by_income[idx_income,1] = np.nanmean(agent_dist_sum_tax_before_tr[agent_mask]) # 平均缴税额现值
        tax_by_income[idx_income,2] = np.nanmean(agent_dist_sum_income_before_tr[agent_mask]) # 平均收入现值
        idx_income+=1

    # 将mean_actions_by_age转换为DataFrame
    tax_by_income = pd.DataFrame(tax_by_income, columns=['有效税率(\%)', '工作期总缴税额', '工作期总收入'], index=[f'{int(start)}-{int(end)}' for start, end in income_bins])
    tax_by_income.index.name = '收入区间'

    '''
    (2) 计算每个生涯平均收入分组的总折现节税额
    '''
    tmp_tax_saving_distf = tax_saving_df.iloc[:tr, 1:].values
    tmp_sum_tax_saving_distf = np.nansum(tmp_tax_saving_distf,axis=0)
    tax_saving_by_income = np.full((len(income_bins),1), np.nan) 
    idx_income = 0
    for start_income, end_income in income_bins:
        agent_mask = (start_income <= agent_meanincome_before_tr) & (agent_meanincome_before_tr < end_income)
        tax_saving_by_income[idx_income] = np.mean(tmp_sum_tax_saving_distf[agent_mask]) # 计算在该收入区间和年龄区间内平均每个agent的总节税额折现值
        idx_income+=1
    tax_saving_by_income = pd.DataFrame(tax_saving_by_income, columns=['总节税额'], index=[f'{int(start)}-{int(end)}' for start, end in income_bins])
    tax_saving_by_income.index.name = '收入区间'

    '''
    (3) 计算每个生涯平均收入分组的养老金购买额度折现值
    '''
    pension_action = pen_amount_df.iloc[:tr, 1:].values
    pension_sum_action = np.nansum(pension_action,axis=0)
    mean_pension_by_income = np.full((len(income_bins),1), np.nan) 
    idx_income = 0
    for start_income, end_income in income_bins:
        agent_mask = (start_income <= agent_meanincome_before_tr) & (agent_meanincome_before_tr < end_income)
        mean_pension_by_income[idx_income] = np.mean(pension_sum_action[agent_mask]) # 计算在该收入区间和年龄区间内平均每个agent的总养老金购买额折现值
        idx_income+=1
    # 将mean_actions_by_age转换为DataFrame
    mean_pension_by_income = pd.DataFrame(mean_pension_by_income, columns=['总养老金购买额'], index=[f'{int(start)}-{int(end)}' for start, end in income_bins])
    mean_pension_by_income.index.name = '收入区间'

    '''
    (4) 计算每个生涯平均收入分组的养老金缴税额现值(每个收入分组指的是agent的终身平均年收入)
    '''
    tmp_pension_tax = pension_tax_df.iloc[tr:, 1:].values
    tmp_sum_pension_tax = np.nansum(tmp_pension_tax,axis=0)
    pension_tax_by_income = np.full((len(income_bins),1), np.nan) 
    idx_income = 0
    for start_income, end_income in income_bins:
        agent_mask = (start_income <= agent_meanincome_before_tr) & (agent_meanincome_before_tr < end_income)
        pension_tax_by_income[idx_income] = np.mean(tmp_sum_pension_tax[agent_mask]) # 计算在该收入区间和年龄区间内平均每个agent的总养老金缴税额折现值
        idx_income+=1
    pension_tax_by_income = pd.DataFrame(pension_tax_by_income, columns=['养老金缴税额'],index=[f'{int(start)}-{int(end)}' for start, end in income_bins])
    pension_tax_by_income.index.name = '收入区间'

   # 编制一个大的表格，将所有表格合并
    combined_data = pd.concat([tax_by_income, tax_saving_by_income, mean_pension_by_income, pension_tax_by_income], axis=1)
    # 增加一列每个收入区间的
    
    '''
    导出为latex表格
    '''
    # 保留小数点后3位
    combined_data.reset_index(drop=False, inplace=True)
    # 仅显示到小数点后3位
    table_tex = combined_data.to_latex(index=False, multicolumn=True, column_format='lcccccc',float_format='%.3f')
    table_tex = table_tex.split(r'\midrule', 1)[1]  # 去除\midrule以前的内容
    # 将表格中的NaN替换为'--'
    table_tex = table_tex.replace('NaN', '--')
    header = r'''
    \begin{table}[htbp]
	\caption{不同收入区间的纳税和购买个人养老金节税结果}
	\label{t2-tax_saving_result}
    \centering
    \begin{threeparttable}
    \begin{tabular}{>{\centering\arraybackslash}p{2cm} >{\centering\arraybackslash}p{2cm} >{\centering\arraybackslash}p{2cm} >{\centering\arraybackslash}p{2cm} >{\centering\arraybackslash}p{2cm} >{\centering\arraybackslash}p{2cm} >{\centering\arraybackslash}p{2cm}} \toprule 收入区间 & 有效税率(\%) & 工作期总缴税额 & 工作期总收入 & 总节税额 & 总养老金购买额 & 养老金缴税额 \\ 
    \midrule
    '''
    ender = r'''\begin{footnotesize}
			注：表中所有值均折现至第一期.
		\end{footnotesize}
        \end{threeparttable}
        \end{table}
        '''
    
    table_tex = header + table_tex.replace('\n', ' ').replace('\\\\', '\\\\ \n') + '\n' + ender # 加入自定义表头和表尾(header和table_tex之间不要有空行)
    # 将table_tex导出为latex文件
    with open('table/t2-tax_saving_result.tex', 'w') as f:
        f.write(table_tex)
    print(table_tex)
    
    





### 表t3: 不同收入区间和不同年龄段DP和RL求解结果比较

In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
from sbx import SAC
# from utils.sac import SAC
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)


# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')

# 将变量处理为终身路径
def var_path(vars, eval_episodes, env):
    max_length = max(len(path) for path in vars)
    var_path_array = np.full((eval_episodes, max_length), np.nan)
    for i, path in enumerate(vars):
        var_path_array[i, :len(path)] = path
    age_array = np.arange(env.get_attr('tb')[0], env.get_attr('tb')[0] + max_length)
    full_var_path = np.column_stack((age_array, var_path_array.T))
    var_path_df = pd.DataFrame(full_var_path,
                                    columns=['年龄'] + [f'个体{i+1}' for i in range(eval_episodes)])
    return var_path_df

import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄
register(
    id='cocco-v0',                                # call it whatever you want
    entry_point='v0_cocco:CoccoEnv', # module_name:class_name
)

log_dir = 'models//sac'

eval_dir = 'eval//'
# model_name = ['v13_pensionfund_penlim0_run7',
#               'v13_pensionfund_penlim1p2_run15',
#               'v13_pensionfund_nopenlim_run7'
#               ]
rl_model_name = 'cocco-v0_run14//RT//RT_4//best_model' # 
rl_vec_env_name = 'cocco-v0_run14//RT//RT_4//best_model_vec_env.pkl'


# model_name = [d for d in os.listdir(log_dir) if os.path.isdir(os.path.join(log_dir, d))]
eval_episodes = 10000
eval_seed =   3687851522
distf = 0.95
import multiprocessing as mp
from functools import partial

def evaluate_model_naive(eval_episodes=eval_episodes, eval_seed=3687851522):
    # 读取cocco模型的A,C,V,gcash


    params = {'pension_limit':1}
    params = None

    env = make_vec_env("cocco-v0",seed=eval_seed, n_envs=1,  env_kwargs={'params': params})

    # 读取matlab dp解
    current_path = os.path.dirname(os.path.abspath(__file__))
    age_group = str(env.get_attr('tb')[0])+'-'+str(env.get_attr('td')[0])
    df_A = pd.read_excel(os.path.join(current_path,'result_cocco_matlab/cocco/A.xlsx'),header=None)
    df_C = pd.read_excel(os.path.join(current_path,'result_cocco_matlab/cocco/C.xlsx'),header=None) 
    df_V = pd.read_excel(os.path.join(current_path,'result_cocco_matlab/cocco/V.xlsx'),header=None)  
    df_gcash = pd.read_excel(os.path.join(current_path,'result_cocco_matlab/cocco/gcash.xlsx'),header=None)
    # 转换为numpy数组
    A = df_A.to_numpy()
    C = df_C.to_numpy()
    V = df_V.to_numpy()
    gcash = df_gcash.to_numpy().squeeze()



    # env = make_vec_env("cocco-dp",seed=1, n_envs=1,  env_kwargs={'params': None})

    action_set = []
    score_history = []
    age = []
    income = []
    consum = []
    temp_income = []
    temp_consum = []
    cash_path = []
    temp_cash = []
    
    reward_history = []

    # from tqdm import tqdm
    pbar = tqdm(total=eval_episodes, desc=f'dp模拟进度')
    for episode in range(eval_episodes):     
        pbar.update(1)
        env.seed(seed=eval_seed+episode)
        obs = env.reset()   
        info = env.reset_infos
        done = False
        score = 0
        episode_actions = []
        temp_reward = []
        while not done:
            # 根据state[0]获取现金
            normalized_cash = np.squeeze(info[0]['normalized_cash']) # 归一化的现金
            cash = obs[0][0]  # 现金
            age_loc =  env.get_attr('age')[0] - env.get_attr('tb')[0] # 获取年龄

            # 根据现金和年龄插值从A中获取action
            action = np.array([np.interp(normalized_cash, gcash, C[:, age_loc]),
                               np.interp(normalized_cash, gcash, A[:, age_loc])])
            if age_loc == np.shape(C)[1]-1: # 如果到达最后一期,则固定消费比例为1（全部消费干净）
                action[0] = normalized_cash
            else:
                action[0] = np.clip(action[0],0,normalized_cash)
            # action[0] = action[0]/cash # 转换为消费比例
            action[0] = action[0]/(normalized_cash) # 消费比例  
            # action[0] = 1 #          
            action[1] = np.clip(action[1],0,1) # 风险资产比例



            obs, reward, done, info = env.step(np.array([action]))
            score += reward * (distf ** len(episode_actions))

            episode_actions.append(action)
            temp_consum.append(info[0]['实际消费'])
            temp_reward.append(reward[0])
            temp_cash.append(info[0]['real_cash'])
            temp_income.append(info[0]['basic_income'])
            # if info[0]['status']=='working':

        age.append(info[0]['age'][0])
        score_history.append(score[0])
        action_set.append(episode_actions)
        reward_history.append(temp_reward)
        consum.append(temp_consum)
        cash_path.append(temp_cash)
        income.append(temp_income)
        temp_consum = []
        temp_cash = []
        temp_income = []

    # 保存评估文件
    # 从模型名中提取标识符（第二个_之后的字符串）
    model_identifier = 'naive'

    os.makedirs(eval_dir + model_identifier + '//', exist_ok=True)


    life_summary = np.array([
        np.mean(age),
        np.mean(score_history),
        np.std(score_history),
    ])

    # =====保存收入路径=====
    income_path_df = var_path(income, eval_episodes, env)
    income_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_income_history.xlsx", index=False)

    # =====保存现金路径=====
    cash_path_df = var_path(cash_path, eval_episodes, env)
    cash_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_cash_history.xlsx", index=False)

     # =====保存消费路径=====
    consum_path_df = var_path(consum, eval_episodes, env)
    consum_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_consum_history.xlsx", index=False)
 
    # 保存action
    max_length = max(len(path) for path in consum)
    unique_age = np.arange(env.get_attr('tb')[0], env.get_attr('td')[0] + 1)[:max_length]
    max_length = max(len(arr) for arr in action_set)
    num_action = np.shape(action_set[0])[1]
    aligned_actions = np.full((max_length, num_action, eval_episodes), np.nan)
    for i, actions in enumerate(action_set):
        aligned_actions[:len(actions), :, i] = actions

    mean_action_by_age = np.nanmean(aligned_actions, axis=2)
    mean_actions = np.column_stack((unique_age, mean_action_by_age))
    mean_actions_df = pd.DataFrame(mean_actions, 
                                    columns=['年龄', '消费', '风险资产'])
    mean_actions_path = eval_dir+ model_identifier + "//" + model_identifier + "_mean_actions.xlsx"
    mean_actions_df.to_excel(mean_actions_path, index=False)

    # 保存score_history
    score_history_df = pd.DataFrame(score_history, columns=['终身效用'])
    score_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_score_history.xlsx", index=False)

    # 保存 reward_history
    reward_history_df = var_path(reward_history, eval_episodes, env)
    reward_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_reward_history.xlsx", index=False)


    life_summary_df = pd.DataFrame([life_summary], 
                                columns=['死亡年龄', '终身效用','终身效用标准差'])
    # 在life_summary_df前面插入模型标识符列
    life_summary_df.insert(0, '模型', model_identifier)
    return aligned_actions, income_path_df, consum_path_df, score_history



def evaluate_model_rl(model_name,vec_name, eval_episodes=eval_episodes, eval_seed=3687851522):
    current_path = os.path.dirname(os.path.abspath(__file__))    

    env = make_vec_env("cocco-v0",seed=1, n_envs=1, env_kwargs={'params': None},monitor_kwargs={'discount_factor':distf})
    

    env = VecNormalize.load(os.path.join(current_path, log_dir,vec_name), env)
    # env.obs_rms.mean
    # env.obs_rms.var
    
    model = SAC.load(os.path.join(current_path, log_dir, model_name), env=env)

    

    # env = make_vec_env("pensionfund-v1",seed=eval_seed, n_envs=1, env_kwargs={'params': None})
    
    # env = VecNormalize.load(os.path.join(current_path, log_dir, idx) + "//eval_env.pkl",env)

    env = make_vec_env("cocco-v0",seed=eval_seed, n_envs=1, env_kwargs={'params':None},monitor_kwargs={'discount_factor': distf})         
    env = VecNormalize(env, norm_obs=False, norm_reward=False) # 评估不需要norm_obs


    idx = model_name.split('//')[0]
    # model = DDPG.load(os.path.join(log_dir, idx) + "//latest_model", env)

    action_set = []
    score_history = []
    age = []
    consum = []
    temp_consum = []
    reward_history = []
    income = []
    temp_income = []
    cash_path = []
    temp_cash = []
    # from tqdm import tqdm
    pbar = tqdm(total=eval_episodes, desc=f'模拟进度 {idx}')
    
    for episode in range(eval_episodes):       
        pbar.update(1)
        env.seed(seed=eval_seed+episode)
        obs = env.reset()    
        done = False
        score = 0
        episode_actions = []
        temp_reward = []
        while not done:
            action, _ = model.predict(model.env.normalize_obs(obs), deterministic=True) 
            # action[0,2] = 0
            cash = obs[0][0] #

            obs, reward, done, info = env.step(action)               
            # reward = -(((cash*action[0][0]))**(1-env.get_attr('gamma')[0]))*10 + 10       
            score += reward * (distf ** len(episode_actions))
            episode_actions.append(info[0]['real_actions'])
            temp_consum.append(info[0]['实际消费'])
            temp_reward.append(reward[0])
            temp_cash.append(info[0]['real_cash'])
            temp_income.append(info[0]['basic_income'])

        score_history.append(score[0])
        reward_history.append(temp_reward)
        action_set.append(episode_actions)
        income.append(temp_income)
        age.append(info[0]['age'][0])
        consum.append(temp_consum)
        cash_path.append(temp_cash)
        temp_consum = []
        temp_cash = []
        temp_income = []

    # 保存评估文件
    # 从模型名中提取标识符（第二个_之后的字符串）
    model_identifier = 'rl_' + idx.split('-', 1)[1] 

    os.makedirs(eval_dir + model_identifier + '//', exist_ok=True)

    # 保存收入路径
    income_path_df = var_path(income, eval_episodes, env)
    income_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_income_history.xlsx", index=False)
    
        # =====保存现金路径=====
    cash_path_df = var_path(cash_path, eval_episodes, env)
    cash_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_cash_history.xlsx", index=False)



    life_summary = np.array([
        np.mean(age),
        np.mean(score_history),
        np.std(score_history),
    ])

    life_summary_df = pd.DataFrame([life_summary], 
                                columns=[ '死亡年龄', '终身效用','终身效用标准差'])

    # =====保存消费路径=====
    consum_path_df = var_path(consum, eval_episodes, env)
    consum_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_consum_history.xlsx", index=False)


    
       # =====保存奖励历史=====
    reward_history_df = var_path(reward_history, eval_episodes, env)
    reward_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_reward_history.xlsx", index=False)

    life_summary = np.array([
        np.mean(age),
        np.mean(score_history),
        np.std(score_history),
    ])

    life_summary_path = eval_dir+ model_identifier + "//" + model_identifier + "_life_summary.xlsx"
    life_summary_df.to_excel(life_summary_path, index=False)

    max_length = max(len(path) for path in consum)
    unique_age = np.arange(env.get_attr('tb')[0], env.get_attr('td')[0] + 1)[:max_length]
    max_length = max(len(arr) for arr in action_set)
    num_action = np.shape(action_set[0])[1]
    aligned_actions = np.full((max_length, num_action, eval_episodes), np.nan)
    for i, actions in enumerate(action_set):
        aligned_actions[:len(actions), :, i] = actions

    mean_action_by_age = np.nanmean(aligned_actions, axis=2)
    mean_actions = np.column_stack((unique_age, mean_action_by_age))
    mean_actions_df = pd.DataFrame(mean_actions, 
                                    columns=['年龄', '消费', '风险资产'])
    mean_actions_path = eval_dir+ model_identifier + "//" + model_identifier + "_mean_actions.xlsx"
    mean_actions_df.to_excel(mean_actions_path, index=False)
    
    #
    
    # 在life_summary_df前面插入模型标识符列
    life_summary_df.insert(0, '模型', model_identifier)
    return aligned_actions, income_path_df, consum_path_df, score_history
    # print(f"模型{idx}评估完毕, 评估次数为{eval_episodes}")
# def evalute_model_erl(idx, eval_episodes=1000, eval_seed=3687851522):

if __name__ == '__main__':   
    tr=46
    aligned_actions_naive, income_path_df_naive, consum_path_df_naive, score_history_naive = evaluate_model_naive()
    aligned_actions_rl, income_path_df_rl, consum_path_df_rl, score_history_rl = evaluate_model_rl(rl_model_name,rl_vec_env_name,eval_episodes=eval_episodes, eval_seed=eval_seed)
    unique_age = income_path_df_naive.iloc[:,0].values
    '''
    DP
    '''
    temp_income_before_tr = income_path_df_naive.iloc[:tr, 1:] # 退休前收入
    agent_meanincome_before_tr = temp_income_before_tr.mean(axis=0) # 退休前平均年收
    temp_consum_dp = consum_path_df_naive.iloc[:, 1:]  # 消费
    '''
    RL
    '''
    temp_consum_rl = consum_path_df_rl.iloc[:, 1:]  # 消费

  # # 循环执行evaluate_model
    tr = 41 # 退休年龄
    distf = 0.95


    # 定义收入分位数和年龄段的分组
    income_quantile_bins = np.array([
        [0, 0.1],
        [0.1, 0.2],    
        [0.2, 0.3],
        [0.3, 0.4],
        [0.4, 0.5],
        [0.5, 0.6],
        [0.6, 0.7],
        [0.7, 0.8],
        [0.8, 0.9],
        [0.9, 0.95],
        [0.95, 1]
    ])
    age_bins = np.array([
        [20, 35],
        [36, 50],
        [51, 61],
        [62, 100],
    ])
    '''
    (1) 计算每个生涯平均收入分组在不同年龄阶段的平均消费值
    '''

    # 循环
    consum_by_income_age_dp = np.full((len(income_quantile_bins),len(age_bins)), np.nan) 
    consum_by_income_age_rl = np.full((len(income_quantile_bins),len(age_bins)), np.nan) 
    idx_income = 0
    idx_age = 0
    idx_age = 0
    for start_age, end_age in age_bins:
        age_mask = (unique_age >= start_age) & (unique_age <= end_age)
        mean_actions_by_income = []
        idx_income = 0
        for start_income, end_income in income_quantile_bins:
            
            agent_mask = (agent_meanincome_before_tr.quantile(start_income) <= agent_meanincome_before_tr) & (agent_meanincome_before_tr < agent_meanincome_before_tr.quantile(end_income))

            consum_by_income_age_dp[idx_income, idx_age] = np.nanmean(temp_consum_dp.loc[age_mask, agent_mask])
            consum_by_income_age_rl[idx_income, idx_age] = np.nanmean(temp_consum_rl.loc[age_mask, agent_mask])
            idx_income+=1
        idx_age+=1

    # 分别连接DP和RL的消费和收入区间

    # 
    consum_by_income_age_dp = pd.DataFrame(consum_by_income_age_dp, columns=[f'{int(start)}-{int(end)}' for start, end in age_bins], index=[f'{float(start)}-{float(end)}' for start, end in income_quantile_bins])
    consum_by_income_age_dp.index.name = '收入区间'
    consum_by_income_age_rl = pd.DataFrame(consum_by_income_age_rl, columns=[f'{int(start)}-{int(end)}' for start, end in age_bins], index=[f'{float(start)}-{float(end)}' for start, end in income_quantile_bins])
    consum_by_income_age_rl.index.name = '收入区间'
 
    '''
    (2) 分别统计每个收入区间在DP和RL下的终身平均效用
    '''
   # 循环
    score_by_income_age_dp = np.full((len(income_quantile_bins),1), np.nan) 
    score_by_income_age_rl = np.full((len(income_quantile_bins),1), np.nan) 
    score_history_array_naive = np.array(score_history_naive)
    score_history_array_rl = np.array(score_history_rl)
    idx_income = 0
    for start_income, end_income in income_quantile_bins:
        agent_mask = (agent_meanincome_before_tr.quantile(start_income) <= agent_meanincome_before_tr) & (agent_meanincome_before_tr < agent_meanincome_before_tr.quantile(end_income))

        score_by_income_age_dp[idx_income] = np.nanmean(score_history_array_naive[agent_mask])
        score_by_income_age_rl[idx_income] = np.nanmean(score_history_array_rl[agent_mask])
        idx_income+=1

    score_by_income_age_dp = pd.DataFrame(score_by_income_age_dp, index=[f'{float(start)}-{float(end)}' for start, end in income_quantile_bins])
    score_by_income_age_dp.index.name = '收入区间'
    score_by_income_age_rl = pd.DataFrame(score_by_income_age_rl, index=[f'{float(start)}-{float(end)}' for start, end in income_quantile_bins])
    score_by_income_age_rl.index.name = '收入区间'    



   # 编制一个大的表格，合并消费和效用，将消费的DP和RL同一收入区间依次排列, 比如RL_30-45, DP_30-45, RL_45-60, DP_45-60,....， 得分排在最后
    combined_data = pd.DataFrame()
    for col in consum_by_income_age_rl.columns:
        combined_data[f'RL消费_{col}'] = consum_by_income_age_rl[col]
        combined_data[f'DP消费_{col}'] = consum_by_income_age_dp[col]
    combined_data['RL得分'] = score_by_income_age_rl
    combined_data['DP得分'] = score_by_income_age_dp
    # 增加一列每个收入区间的
    '''
    导出为latex表格
    '''
    combined_data.reset_index(drop=False, inplace=True)
    # 仅显示到小数点后3位
    table_tex = combined_data.to_latex(index=False, multicolumn=True, column_format='c' * len(combined_data.columns), float_format='%.3f')
    table_tex = table_tex.split(r'\midrule', 1)[1]  # 去除\midrule以前的内容
    # 将表格中的NaN替换为'--'
    table_tex = table_tex.replace('NaN', '--')
    header = r'''
    \begin{table}[htbp]
	\caption{不同收入区间的DP和RL消费和效用结果比较}
	\label{t3-action_reward_by_income_age_comparison}
    \centering
    \begin{threeparttable}
    \begin{tabular}{@{}ccccccccccc@{}}
    \toprule
    \multirow{3}{*}{年收入区间} & \multicolumn{10}{c}{年龄区间}                                                                                                               \\ \cmidrule(l){2-11} 
                        & \multicolumn{2}{c}{20-35} & \multicolumn{2}{c}{36-50} & \multicolumn{2}{c}{51-61} & \multicolumn{2}{c}{62-100} & \multicolumn{2}{c}{终身} \\ \cmidrule(l){2-11} 
                        & RL          & DP          & RL          & DP          & RL          & DP          & RL           & DP          & RL         & DP        \\ \midrule
    '''
    ender = r'''\begin{footnotesize}
			注：
		\end{footnotesize}
        \end{threeparttable}
        \end{table}
        '''
    
    table_tex = header + table_tex.replace('\n', ' ').replace('\\\\', '\\\\ \n') + '\n' + ender # 加入自定义表头和表尾(header和table_tex之间不要有空行)
    # 将table_tex导出为latex文件
    with open('table/t3-action_reward_by_income_age_comparison.tex', 'w') as f:
        f.write(table_tex)
    print(table_tex)




### 表t4: 不同养老金约束的CEQ

In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
from sbx import SAC
# from utils.sac import SAC
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
# import matplotlib
# print(matplotlib.matplotlib_fname())
# raise

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)


# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')

# 将变量处理为终身路径
def var_path(vars, eval_episodes, env):
    max_length = max(len(path) for path in vars)
    var_path_array = np.full((eval_episodes, max_length), np.nan)
    for i, path in enumerate(vars):
        var_path_array[i, :len(path)] = path
    age_array = np.arange(env.get_attr('tb')[0], env.get_attr('tb')[0] + max_length)
    full_var_path = np.column_stack((age_array, var_path_array.T))
    var_path_df = pd.DataFrame(full_var_path,
                                    columns=['年龄'] + [f'个体{i+1}' for i in range(eval_episodes)])
    return var_path_df

import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄
register(
    id='pensionfund-v5',                                # call it whatever you want
    entry_point='v5_pensionfund:PensionFundEnv', # module_name:class_name
)
log_dir = '../autodl-tmp/models//sac//'

eval_dir = 'eval//'
# model_name = ['v13_pensionfund_penlim0_run7',
#               'v13_pensionfund_penlim1p2_run15',
#               'v13_pensionfund_nopenlim_run7'
#               ]
rl_model_name = 'pensionfund-v5_run50//RT//RT_3//RT_1//best_model' # 
rl_vec_env_name = 'pensionfund-v5_run50//RT//RT_3//RT_1//best_model_vec_env.pkl'


# model_name = [d for d in os.listdir(log_dir) if os.path.isdir(os.path.join(log_dir, d))]
eval_episodes = 200
eval_seed =   3687851522
distf = 0.95
import multiprocessing as mp
from functools import partial

def evaluate_model_naive(eval_episodes=10000, eval_seed=3687851522):
    # 读取cocco模型的A,C,V,gcash


    params = {'pension_limit':1}
    params = None

    env = make_vec_env("pensionfund-v5",seed=eval_seed, n_envs=1,  env_kwargs={'params': params})

    # 读取matlab dp解
    current_path = os.path.dirname(os.path.abspath(__file__))
    age_group = str(env.get_attr('tb')[0])+'-'+str(env.get_attr('td')[0])
    df_A = pd.read_excel(os.path.join(current_path,'result_cocco_matlab',age_group,'A.xlsx'),header=None)
    df_C = pd.read_excel(os.path.join(current_path,'result_cocco_matlab',age_group,'C.xlsx'),header=None) 
    df_V = pd.read_excel(os.path.join(current_path,'result_cocco_matlab',age_group,'V.xlsx'),header=None)  
    df_gcash = pd.read_excel(os.path.join(current_path,'result_cocco_matlab',age_group,'gcash.xlsx'),header=None)
    # 转换为numpy数组
    A = df_A.to_numpy()
    C = df_C.to_numpy()
    V = df_V.to_numpy()
    gcash = df_gcash.to_numpy().squeeze()



    # env = make_vec_env("cocco-dp",seed=1, n_envs=1,  env_kwargs={'params': None})

    action_set = []
    score_history = []
    age = []
    wealth = []
    tmp_wealth = []

    # from tqdm import tqdm
    for episode in range(eval_episodes):     
        env.seed(seed=eval_seed+episode)
        obs = env.reset()   
        info = env.reset_infos
        done = False
        score = 0
        episode_actions = []
        while not done:
            # 根据state[0]获取现金
            normalized_cash = np.squeeze(info[0]['normalized_cash']) # 归一化的现金
            cash = obs[0][0]  # 现金
            age_loc =  env.get_attr('age')[0] - env.get_attr('tb')[0] # 获取年龄

            # 根据现金和年龄插值从A中获取action
            action = np.array([np.interp(normalized_cash, gcash, C[:, age_loc]),
                               np.interp(normalized_cash, gcash, A[:, age_loc]),0,0])
            if age_loc == np.shape(C)[1]-1: # 如果到达最后一期,则固定消费比例为1（全部消费干净）
                action[0] = normalized_cash
            else:
                action[0] = np.clip(action[0],0,normalized_cash)
            # action[0] = action[0]/cash # 转换为消费比例
            action[0] = action[0]/(normalized_cash) # 消费比例  
            # action[0] = 1 #          
            action[1] = np.clip(action[1],0,1) # 风险资产比例



            obs, reward, done, info = env.step(np.array([action]))
            score += reward * (distf ** len(episode_actions))
            episode_actions.append(action)
            tmp_wealth.append(obs[0][0]*np.exp(1))
        action_set.append(episode_actions)
        age.append(info[0]['age'][0])
        score_history.append(score[0])

    max_length = max(len(path) for path in action_set)
    unique_age = np.arange(env.get_attr('tb')[0], env.get_attr('td')[0] + 1)[:max_length]
    num_action = np.shape(action_set[0])[1]
    aligned_actions = np.full((max_length, num_action, eval_episodes), np.nan)
    for i, actions in enumerate(action_set):
        aligned_actions[:len(actions), :, i] = actions


    return np.mean(score_history)




# def evalute_model_erl(idx, eval_episodes=1000, eval_seed=3687851522):


def evaluate_model_rl(model_name,vec_name, eval_episodes=1000, eval_seed=3687851522,params=None):
    current_path = os.path.dirname(os.path.abspath(__file__))    
    # try:
    #     with open(os.path.join(current_path, log_dir, model_name.split('//')[0]) +"//params.json", 'r') as file:
    #         params = json.load(file)   
    # except:
    # if 'cocco' in idx:
    #     env = make_vec_env("cocco-rl-norm",seed=1, n_envs=1, env_kwargs={'params': params})
    # else:
    # env = make_vec_env("pensionfund-v3",seed=1, n_envs=1, env_kwargs={'params': params})````````
    env = make_vec_env("pensionfund-v5",seed=1, n_envs=1, env_kwargs={'params': params},\
                            monitor_kwargs={'discount_factor':distf})
    

    env = VecNormalize.load(os.path.join(current_path, log_dir,vec_name), env)
    # env.obs_rms.mean
    # env.obs_rms.var
    
    model = SAC.load(os.path.join(current_path, log_dir, model_name), env=env)

    

    # env = make_vec_env("pensionfund-v1",seed=eval_seed, n_envs=1, env_kwargs={'params': None})
    
    # env = VecNormalize.load(os.path.join(current_path, log_dir, idx) + "//eval_env.pkl",env)

    env = make_vec_env("pensionfund-v5",seed=eval_seed, n_envs=1, env_kwargs={'params':params},\
                            monitor_kwargs={'discount_factor': distf})         
    env = VecNormalize(env, norm_obs=False, norm_reward=False) # 评估不需要norm_obs


    idx = model_name.split('//')[0]
    # model = DDPG.load(os.path.join(log_dir, idx) + "//latest_model", env)

    action_set = []
    age = []
    score_history = []
    income = []
    temp_income = []
    # from tqdm import tqdm
    
    for episode in range(eval_episodes):       
        env.seed(seed=eval_seed+episode)
        obs = env.reset()    
        done = False
        score = 0
        episode_actions = []
        while not done:
            action, _ = model.predict(model.env.normalize_obs(obs), deterministic=True) 
            # action[0,2] = 0

            obs, reward, done, info = env.step(action)               
            # reward = -(((cash*action[0][0]))**(1-env.get_attr('gamma')[0]))*10 + 10       
            score += reward * (distf ** len(episode_actions))
            episode_actions.append(info[0]['real_actions'])
            # if info[0]['status']=='working':
            try:
                tmp = info[0]['养老金实际余额'][0] # 退休后养老金实际余额
            except:
                tmp = obs[0][2]*np.exp(1) # 退休前养老金实际余额
            temp_income.append(info[0]['basic_income'])
        action_set.append(episode_actions)
        age.append(info[0]['age'][0])
        score_history.append(score[0])
        income.append(temp_income)
        temp_income = []

    max_length = max(len(path) for path in action_set)
    unique_age = np.arange(env.get_attr('tb')[0], env.get_attr('td')[0] + 1)[:max_length]
    num_action = np.shape(action_set[0])[1]
    aligned_actions = np.full((max_length, num_action, eval_episodes), np.nan)
    for i, actions in enumerate(action_set):
        aligned_actions[:len(actions), :, i] = actions
    
    income_path_df = var_path(income, eval_episodes, env)

    return score_history,env,income_path_df

if __name__ == '__main__':


    # 创建进程池
    # pool = mp.Pool()
    # # 循环执行evaluate_model
    from tqdm import tqdm
    navie_score = evaluate_model_naive(eval_episodes=eval_episodes, eval_seed=eval_seed)
    params = {'pension_limit':0.3}
    baseline_score,env,income_path_df= evaluate_model_rl(rl_model_name,rl_vec_env_name,eval_episodes=eval_episodes, eval_seed=eval_seed,params=params)


    # 计算Consumption-equivalent variations
    distf = env.get_attr('beta')[0]
    gamma = env.get_attr('gamma')[0]
    period = env.get_attr('tn')[0]
    tb = env.get_attr('tb')[0]
    tr = env.get_attr('tr')[0]

    baseline_c = (-np.nanmean(baseline_score) / np.sum(np.ones(period)*(distf ** np.arange(period))))**(1/(1-gamma))
    
    '''
    naive解
    '''
    naive_c = (-navie_score / np.sum(np.ones(period)*(distf ** np.arange(period))))**(1/(1-gamma)) # CE
    cev = {'无养老金模型':(baseline_c - naive_c) / naive_c}

    penlimit_list = [0.6,1.2,2,3,5]
    income_quantile_bins = np.array([
    [0, 1]
    ])
    temp_income_before_tr = income_path_df.iloc[:tr, 1:] # 退休前收入
    agent_meanincome_before_tr = temp_income_before_tr.mean(axis=0) # 退休前平均
    
    cev = pd.DataFrame(index=[f'{float(start)}-{float(end)}' for start, end in income_quantile_bins],columns=[f'penlimit_{penlimit}万' for penlimit in penlimit_list])
    idx_income = 0
    idx_penlimit = 0
    from tqdm import tqdm
    for penlimit in tqdm(penlimit_list):
        params = {'pension_limit_yuan':penlimit}
        score_history,_,_= evaluate_model_rl(rl_model_name,rl_vec_env_name,eval_episodes=eval_episodes, eval_seed=eval_seed,params=params)
          
        for start_income, end_income in income_quantile_bins:


            agent_mask = (agent_meanincome_before_tr.quantile(start_income) <= agent_meanincome_before_tr) & (agent_meanincome_before_tr < agent_meanincome_before_tr.quantile(end_income))
            mean_score = np.nanmean(np.array(score_history)[agent_mask])
            pen01_c = (-mean_score / np.sum(np.ones(period)*(distf ** np.arange(period))))**(1/(1-gamma))

            # 计算baseline在
            baseline_mean_score = np.nanmean(np.array(baseline_score)[agent_mask])
            baseline_c = (-baseline_mean_score / np.sum(np.ones(period)*(distf ** np.arange(period))))**(1/(1-gamma))

            cev.loc[f'{float(start_income)}-{float(end_income)}',f'penlimit_{penlimit}万'] = (baseline_c - pen01_c) *100 / pen01_c

    # 保存cev
    print(cev)



    raise
    
    





## 生成评估数据

In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
# from stable_baselines3 import SAC
from utils.sac import SAC
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)


# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')

# 将变量处理为终身路径
def var_path(vars, eval_episodes, env):
    max_length = max(len(path) for path in vars)
    var_path_array = np.full((eval_episodes, max_length), np.nan)
    for i, path in enumerate(vars):
        var_path_array[i, :len(path)] = path
    age_array = np.arange(env.get_attr('tb')[0], env.get_attr('tb')[0] + max_length)
    full_var_path = np.column_stack((age_array, var_path_array.T))
    var_path_df = pd.DataFrame(full_var_path,
                                    columns=['年龄'] + [f'个体{i+1}' for i in range(eval_episodes)])
    return var_path_df

import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄
register(
    id='pensionfund-v3',                                # call it whatever you want
    entry_point='v3_pensionfund:PensionFundEnv', # module_name:class_name
)
register(
    id='cocco-dp',
    entry_point='v3_cocco:CoccoEnv',
)
register(
    id='cocco-rl-norm',
    entry_point='cocco_env_rl_norm:CoccoEnv',
)

log_dir = 'models//sac'

eval_dir = 'eval//'
# model_name = ['v13_pensionfund_penlim0_run7',
#               'v13_pensionfund_penlim1p2_run15',
#               'v13_pensionfund_nopenlim_run7'
#               ]
rl_model_name = [
    'cocco-rl-norm_cocco_run97//best_model', # 基准的无养老金模型
    # 'pensionfund-v3_run62//CT//best_model', # penlimit=1.2
    # 'pensionfund-v3_run74//best_model'
              ]

# model_name = [d for d in os.listdir(log_dir) if os.path.isdir(os.path.join(log_dir, d))]
eval_episodes = 200
eval_seed =  3687851522
distf = 0.95

import multiprocessing as mp
from functools import partial

def evaluate_model_naive(eval_episodes=10000, eval_seed=3687851522):
    # 读取cocco模型的A,C,V,gcash
    current_path = os.path.dirname(os.path.abspath(__file__))
    df_A = pd.read_excel(os.path.join(current_path,'result_cocco_matlab','A.xlsx'),header=None)
    df_C = pd.read_excel(os.path.join(current_path,'result_cocco_matlab','C.xlsx'),header=None) 
    df_V = pd.read_excel(os.path.join(current_path,'result_cocco_matlab','V.xlsx'),header=None)  
    df_gcash = pd.read_excel(os.path.join(current_path,'result_cocco_matlab','gcash.xlsx'),header=None)
    # 转换为numpy数组
    A = df_A.to_numpy()
    C = df_C.to_numpy()
    V = df_V.to_numpy()
    gcash = df_gcash.to_numpy().squeeze()


    params = {'pension_limit': 1.2}
    # params = None

    env = make_vec_env("pensionfund-v3",seed=1, n_envs=1,  env_kwargs={'params': params})
    # env = make_vec_env("cocco-dp",seed=1, n_envs=1,  env_kwargs={'params': None})

    action_set = []
    score_history = []
    age = []
    income = []
    consum = []
    tmp_income = []
    temp_consum = []
    
    reward_history = []

    # from tqdm import tqdm
    pbar = tqdm(total=eval_episodes, desc=f'dp模拟进度')
    for episode in range(eval_episodes):     
        pbar.update(1)
        env.seed(seed=eval_seed+episode)
        obs = env.reset()   
        info = env.reset_infos
        done = False
        score = 0
        episode_actions = []
        temp_reward = []
        while not done:
            # 根据state[0]获取现金
            normalized_cash = np.squeeze(info[0]['normalized_cash']) # 归一化的现金
            cash = obs[0][0]  # 现金
            age_loc =  env.get_attr('age')[0] - env.get_attr('tb')[0] # 获取年龄

            # 根据现金和年龄插值从A中获取action
            action = np.array([np.interp(normalized_cash, gcash, C[:, age_loc]),
                               np.interp(normalized_cash, gcash, A[:, age_loc]),
                               0,0])
            if age_loc == np.shape(C)[1]-1: # 如果到达最后一期,则固定消费比例为1（全部消费干净）
                action[0] = normalized_cash
            else:
                action[0] = np.clip(action[0],0,normalized_cash)
            # action[0] = action[0]/cash # 转换为消费比例
            action[0] = action[0]/(normalized_cash) # 消费比例           
            action[1] = np.clip(action[1],0,1) # 风险资产比例



            obs, reward, done, info = env.step(np.array([action]))
            score += reward * (distf ** len(episode_actions))

            episode_actions.append(action)
            temp_consum.append(cash*action[0])
            temp_reward.append(reward[0])
            # if info[0]['status']=='working':
            tmp_income.append(info[0]['basic_income'])

        age.append(info[0]['age'][0])
        score_history.append(score[0])
        action_set.append(episode_actions)
        reward_history.append(temp_reward)
        consum.append(temp_consum)
        income.append(tmp_income)

        tmp_income = []
        temp_consum = []


    life_path_detailed = pd.DataFrame({
        '死亡年龄': age,
        '工作期平均收入(万)': income,
        '终身效用': score_history,
    })
    # 保存评估文件
    # 从模型名中提取标识符（第二个_之后的字符串）
    model_identifier = 'naive'

    os.makedirs(eval_dir + model_identifier + '//', exist_ok=True)


    
    # consum_path_array = np.full((eval_episodes, max_length), np.nan)
    # for i, path in enumerate(consum):
    #     consum_path_array[i, :len(path)] = path
    # age_array = np.arange(env.get_attr('tb')[0], env.get_attr('tb')[0] + max_length)
    # full_consum_path = np.column_stack((age_array, consum_path_array.T))
    # consum_path_df = pd.DataFrame(full_consum_path,
    #                                 columns=['年龄'] + [f'个体{i+1}' for i in range(eval_episodes)])
    # consum_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_consum_path.xlsx", index=False)
        # =====保存消费路径=====
    consum_path_df = var_path(consum, eval_episodes, env)
    consum_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_consum_history.xlsx", index=False)

    life_path_detailed_path = eval_dir+ model_identifier + "//" + model_identifier + "_life_path_detailed.xlsx"
    life_path_detailed.to_excel(life_path_detailed_path, index=False)
    
        # 保存收入历史
    income_history_df = var_path(income, eval_episodes, env)
    income_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_income_history.xlsx", index=False)

    income = income_history_df.iloc[:env.get_attr('tr')[0],1:]
    income_summary_df = pd.DataFrame([np.array([
        np.mean(income),
        np.nanstd(income.values.flatten()),
        np.min(income),
        np.max(income),
    ])],columns=['工作期平均收入(w)', '标准差','最小值','最大值'])
    print(income_summary_df)
       # =====保存奖励历史=====
    reward_history_df = var_path(reward_history, eval_episodes, env)
    reward_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_reward_history.xlsx", index=False)

    life_summary = np.array([
        np.mean(age),
        np.mean(score_history),
        np.std(score_history),
    ])


    life_summary_df = pd.DataFrame([life_summary], 
                                columns=['死亡年龄', '终身效用','终身效用标准差'])
    life_summary_path = eval_dir+ model_identifier + "//" + model_identifier + "_life_summary.xlsx"
    life_summary_df.to_excel(life_summary_path, index=False)

    # reward_history_array = np.full((eval_episodes, max_length), np.nan)
    # for i, path in enumerate(reward_history):
    #     reward_history_array[i, :len(path)] = path
    # full_reward_history = np.column_stack((age_array, reward_history_array.T))
    # reward_history_df = pd.DataFrame(full_reward_history,
    #                                 columns=['年龄'] + [f'个体{i+1}' for i in range(eval_episodes)])
    # reward_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_reward_history.xlsx", index=False)
    max_length = max(len(path) for path in consum)
    unique_age = np.arange(env.get_attr('tb')[0], env.get_attr('td')[0] + 1)[:max_length]
    max_length = max(len(arr) for arr in action_set)
    num_action = np.shape(action_set[0])[1]
    aligned_actions = np.full((max_length, num_action, eval_episodes), np.nan)
    for i, actions in enumerate(action_set):
        aligned_actions[:len(actions), :, i] = actions

    mean_action_by_age = np.nanmean(aligned_actions, axis=2)
    mean_actions = np.column_stack((unique_age, mean_action_by_age))
    mean_actions_df = pd.DataFrame(mean_actions, 
                                    columns=['年龄', '消费', '风险资产','个人养老金购买比例','个人养老金风险资产比例'])
    mean_actions_path = eval_dir+ model_identifier + "//" + model_identifier + "_mean_actions.xlsx"
    mean_actions_df.to_excel(mean_actions_path, index=False)


  
    # 在life_summary_df前面插入模型标识符列
    life_summary_df.insert(0, '模型', model_identifier)
    return life_summary_df

    

def evaluate_model_rl(idx, eval_episodes=1000, eval_seed=3687851522):
    current_path = os.path.dirname(os.path.abspath(__file__))    
    with open(os.path.join(current_path, log_dir, idx.split('//')[0]) +"//params.json", 'r') as file:
        params = json.load(file)   
    if 'cocco' in idx:
        env = make_vec_env("cocco-rl-norm",seed=1, n_envs=1, env_kwargs={'params': params})
    else:
        env = make_vec_env("pensionfund-v3",seed=1, n_envs=1, env_kwargs={'params': params})
    
    env = VecNormalize.load(os.path.join(current_path, log_dir,idx.split('//')[0]) + "//vec_env.pkl", env)
    model = SAC.load(os.path.join(current_path, log_dir, idx), env=env)

    

    # env = make_vec_env("pensionfund-v1",seed=eval_seed, n_envs=1, env_kwargs={'params': None})
    
    # env = VecNormalize.load(os.path.join(current_path, log_dir, idx) + "//eval_env.pkl",env)

    env = make_vec_env("pensionfund-v3",seed=eval_seed, n_envs=1, env_kwargs={'params':params},\
                            monitor_kwargs={'discount_factor': distf})         
    env = VecNormalize(env, norm_obs=False, norm_reward=False) # 评估不需要norm_obs


    idx = idx.split('//')[0]
    # model = DDPG.load(os.path.join(log_dir, idx) + "//latest_model", env)

    action_set = []
    score_history = []
    age = []
    income = []
    consum = []
    tmp_income = []
    temp_consum = []
    reward_history = []
    # from tqdm import tqdm
    pbar = tqdm(total=eval_episodes, desc=f'模拟进度 {idx}')
    
    for episode in range(eval_episodes):       
        pbar.update(1)
        env.seed(seed=eval_seed+episode)
        obs = env.reset()    
        done = False
        score = 0
        episode_actions = []
        temp_reward = []
        while not done:
            if 'cocco' in idx:
                action , _ = model.predict(model.env.normalize_obs(obs[:,[0,1,3]]), deterministic=True) 
                action = np.array([np.append(action, [0, 0])])
            else:
                action, _ = model.predict(model.env.normalize_obs(obs), deterministic=True) 
            cash = obs[0][0] #

            obs, reward, done, info = env.step(action)               
            # reward = -(((cash*action[0][0]))**(1-env.get_attr('gamma')[0]))*10 + 10       
            score += reward * (distf ** len(episode_actions))
            episode_actions.append(info[0]['real_actions'])
            temp_consum.append(cash*action[0][0])
            temp_reward.append(reward[0])
            if info[0]['status']=='working':
                tmp_income.append(info[0]['basic_income'])
        
        score_history.append(score)
        reward_history.append(temp_reward)
        action_set.append(episode_actions)
        # norm_age = info[0]['state'][4]
        # age.append(np.round((norm_age + 1) * (params['max_age'] - params['born_age']) / 2 + params['born_age'], 1) - 1)
        age.append(info[0]['age'][0])

        consum.append(temp_consum)
        income.append(tmp_income)

        tmp_income = []
        temp_consum = []


    life_path_detailed = pd.DataFrame({
        '死亡年龄': age,
        '工作期平均收入(万)': income,
        '终身效用': score_history,
    })
    # 保存评估文件
    # 从模型名中提取标识符（第二个_之后的字符串）
    model_identifier = 'rl_' + idx.split('-', 1)[1] 

    os.makedirs(eval_dir + model_identifier + '//', exist_ok=True)


    
    # consum_path_array = np.full((eval_episodes, max_length), np.nan)
    # for i, path in enumerate(consum):
    #     consum_path_array[i, :len(path)] = path
    # age_array = np.arange(env.get_attr('tb')[0], env.get_attr('tb')[0] + max_length)
    # full_consum_path = np.column_stack((age_array, consum_path_array.T))
    # consum_path_df = pd.DataFrame(full_consum_path,
    #                                 columns=['年龄'] + [f'个体{i+1}' for i in range(eval_episodes)])
    # consum_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_consum_path.xlsx", index=False)

        # =====保存消费路径=====
    consum_path_df = var_path(consum, eval_episodes, env)
    consum_path_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_consum_history.xlsx", index=False)

    life_path_detailed_path = eval_dir+ model_identifier + "//" + model_identifier + "_life_path_detailed.xlsx"
    life_path_detailed.to_excel(life_path_detailed_path, index=False)
    
        # 保存收入历史
    income_history_df = var_path(income, eval_episodes, env)
    income_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_income_history.xlsx", index=False)
    income = income_history_df.iloc[:env.get_attr('tr')[0],1:]
    income_summary_df = pd.DataFrame([np.array([
        np.mean(income),
        np.nanstd(income.values.flatten()),
        np.min(income),
        np.max(income),
    ])],columns=['工作期平均收入(w)', '标准差','最小值','最大值'])
    print(income_summary_df)
       # =====保存奖励历史=====
    reward_history_df = var_path(reward_history, eval_episodes, env)
    reward_history_df.to_excel(eval_dir+ model_identifier + "//" + model_identifier + "_reward_history.xlsx", index=False)

    life_summary = np.array([
        np.mean(age),
        np.mean(score_history),
        np.std(score_history),
    ])

    life_path_detailed_path = eval_dir+ model_identifier + "//" + model_identifier + "_life_path_detailed.xlsx"
    life_path_detailed.to_excel(life_path_detailed_path, index=False)


    life_summary_df = pd.DataFrame([life_summary], 
                                columns=[ '死亡年龄', '终身效用','终身效用标准差'])
    life_summary_path = eval_dir+ model_identifier + "//" + model_identifier + "_life_summary.xlsx"
    life_summary_df.to_excel(life_summary_path, index=False)

    max_length = max(len(path) for path in consum)
    unique_age = np.arange(env.get_attr('tb')[0], env.get_attr('td')[0] + 1)[:max_length]
    max_length = max(len(arr) for arr in action_set)
    num_action = np.shape(action_set[0])[1]
    aligned_actions = np.full((max_length, num_action, eval_episodes), np.nan)
    for i, actions in enumerate(action_set):
        aligned_actions[:len(actions), :, i] = actions

    mean_action_by_age = np.nanmean(aligned_actions, axis=2)
    mean_actions = np.column_stack((unique_age, mean_action_by_age))
    mean_actions_df = pd.DataFrame(mean_actions, 
                                    columns=['年龄', '消费', '风险资产','个人养老金购买比例','个人养老金风险资产比例'])
    mean_actions_path = eval_dir+ model_identifier + "//" + model_identifier + "_mean_actions.xlsx"
    mean_actions_df.to_excel(mean_actions_path, index=False)


  
    # 在life_summary_df前面插入模型标识符列
    life_summary_df.insert(0, '模型', model_identifier)
    return life_summary_df
    # print(f"模型{idx}评估完毕, 评估次数为{eval_episodes}")


if __name__ == '__main__':


    # 创建进程池
    # pool = mp.Pool()
    
    # # 循环执行evaluate_model
    from tqdm import tqdm
    life_summary_naive = evaluate_model_naive(eval_episodes=eval_episodes, eval_seed=eval_seed)
    print(life_summary_naive.to_string(index=False))
    # life_summary_rl = []
    # for model_name in tqdm(rl_model_name, desc='总进度'):
    #     result = evaluate_model_rl(model_name, eval_episodes=eval_episodes, eval_seed=eval_seed)
    #     life_summary_rl.append(result)
    # life_summary = pd.concat([life_summary_naive, *life_summary_rl], ignore_index=True)
    # pd.set_option('display.float_format', lambda x: '{:.8f}'.format(x))
    # pd.set_option('display.colheader_justify', 'center')
    # pd.set_option('display.unicode.ambiguous_as_wide', True)
    # pd.set_option('display.unicode.east_asian_width', True)
    # print(life_summary.to_string(index=False))




## 每期score分布情况：确定reward怎么设置

In [ ]:
from typing import Callable
import json
import gymnasium as gym
import numpy as np
import os
# from stable_baselines3 import DDPG
# from stable_baselines3 import TD3
# from stable_baselines3 import PPO
# from stable_baselines3 import SAC
from sbx import DDPG, DQN, PPO, SAC, TD3, TQC, CrossQ
from utils.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize
from gymnasium.envs.registration import register
import pandas as pd
import matplotlib.pyplot as plt
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# get current directory of the current .ipynb 
cdir = os.path.abspath('.') # 获取当前目录
pardir = os.path.abspath(os.path.join(os.getcwd(), "../")) # 获取上级目录
dbdir = os.path.abspath(os.path.join(os.getcwd(), "../..")) # 获取上上级目录(为了读取美股数据库数据)
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签  
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
# import torch 
# torch.autograd.set_detect_anomaly(True)
import warnings
warnings.filterwarnings('ignore')


import sys
cdir = os.path.abspath('.') # 获取当前目录
# print(sys.path)
# 状态空间：现金余额，个人养老金账户余额，永久收入冲击，年龄

log_dir = 'models//cocco_sac'

eval_dir = 'eval//'
rl_model_name = [
            #   'cocco-rl_cocco_run225//best_model_67', # 无随机死亡最佳
            # 'cocco-rl_cocco_run231//best_model_67', # 有随机死亡最佳 
            # 'cocco-rl_cocco_run242//best_model_618'
            'cocco-rl-norm_cocco_run11//best_model'
              ]




if __name__ == '__main__':
        # 获取当前所在路径中"博士论文"所在目录
    current_path = os.getcwd()
    while os.path.basename(current_path) != "博士论文":
        current_path = os.path.dirname(current_path)
        if current_path == os.path.dirname(current_path):  # 已到达根目录仍未找到
            raise Exception("未找到'博士论文'目录")
    thesis_dir = current_path + "\\文字\\2-与传统方法比较\\fig\\"

    # =====读取dp和rl model的score_history======
    gamma = 3.84
    utility_path = {}
    income_path = {}
    model_identifier = 'dp'
    dp_path = eval_dir + model_identifier + "//" + model_identifier + "_income_history.xlsx"
    income_path['DP'] = pd.read_excel(dp_path)

    dp_path = eval_dir + model_identifier + "//" + model_identifier + "_consum_history.xlsx"
    utility_path['DP'] = pd.read_excel(dp_path)
    # utility_path['DP'].iloc[:,1:] = utility_path['DP'].iloc[:,1:]**(1-3.84)/(1-3.84)
    

    # temp[temp<-5] = -5


    # 创建包含两个子图的图形
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 6))

    # 子图1：绘制收入直方图
    # 剔除nan值
    temp = income_path['DP'].iloc[:,1:].values
    temp = temp[~np.isnan(temp)]
    ax1.hist(temp, bins=200, weights=np.ones(len(temp)) / len(temp), alpha=0.7, color='r', label='概率分布')
    ax1.set_xlabel('收入值', fontsize=12)
    ax1.set_ylabel('出现比例', fontsize=12)
    ax1.set_title('动态规划模型收入的概率分布', fontsize=14)
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=10)

    
    # 子图2：绘制消费直方图  
    # 剔除nan值
    temp = utility_path['DP'].iloc[:,1:].values
    temp = temp[~np.isnan(temp)]
    ax2.hist(temp, bins=200, weights=np.ones(len(temp)) / len(temp), alpha=0.7, color='r', label='概率分布')
    ax2.set_xlabel('消费值', fontsize=12)
    ax2.set_ylabel('出现比例', fontsize=12)
    ax2.set_title('动态规划模型消费的概率分布', fontsize=14)
    ax2.grid(True, alpha=0.3)
    ax2.legend(fontsize=10)

    # 子图3：绘制效用直方图
    utility_path['DP'].iloc[:,1:] = np.log(utility_path['DP'].iloc[:,1:]) # 需要调试的效用函数
    temp = utility_path['DP'].iloc[:,1:].values
    temp = temp[~np.isnan(temp)]
    # temp = temp[temp>-3]
    # temp[temp<-2]=-2
    
    ax3.hist(temp, bins=200, weights=np.ones(len(temp)) / len(temp), alpha=0.7, color='b', label='概率分布')
    ax3.set_xlabel('效用值', fontsize=12)
    ax3.set_ylabel('出现比例', fontsize=12)
    ax3.set_title('动态规划模型效用的概率分布', fontsize=14)
    ax3.grid(True, alpha=0.3)
    ax3.legend(fontsize=10)

    plt.tight_layout()
    plt.show()
